<a href="https://colab.research.google.com/github/Thomas-Fabbris/parallel-computing-polimi/blob/main/CUDA_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CUDA**

Exploiting NVIDIA GPUs for general-purpose computing.


## **Setup**

In [ ]:
%%capture
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

In [ ]:
!mkdir /home/cuda
%cd /home/cuda

/home/cuda


## **Glossary**

**Thread**: one running instance of the kernel, with its own execution context.<br>
**Warp**: group of threads that are scheduled together on a streaming multiprocessor (SM) and get SIMD (or, as Nvidia calls it, SIMT) execution when their program counters align.<br>
**Block**: group of threads that get assigned to the same streaming multiprocessor, sharing shared memory and synchronization. A block internally consists of one or more warps.<br>
**Grid**: group of blocks that defines the full launch configuration of a kernel.

*Note: Warps are decided by the hardware, blocks and grid are user defined.*

---

Full math for a thread's ID inside a block:
```
thread_id = threadIdx.x +
            threadIdx.y * blockDim.x +
            threadIdx.z * blockDim.x * blockDim.y
```

ID of the block the current thread is part of:
```
block_id = blockIdx.x +
           gridDim.x * blockIdx.y +
           gridDim.x * gridDim.y * blockIdx.z
```

Absolute (grid-level) thread ID and position:
```
global_thread_id = block_id * (blockDim.x * blockDim.y * blockDim.z) + thread_id

global_x_pos = blockIdx.x * blockDim.x + threadIdx.x
global_y_pos = blockIdx.y * blockDim.y + threadIdx.y
global_z_pos = blockIdx.z * blockDim.z + threadIdx.z
```

Thread idx inside a warp and warp ID in a block:
```
idx_in_warp = thread_id % warpSize

warp_id = thread_id / warpSize
```
Dimensions are projected into a linearized row-major (x before y before z) order before partitioning into warps.

*Note: for a block whose size is not a multiple of warpSize, the last warp will be padded with extra threads to fill up the warpSize-thread positions. For example, with warpSize = 32, if a block has 48 threads, it will be partitioned into two warps, and its second warp will be padded with 16 extra thread indices.*

*Note: a nice name for this is **hierarchical indexing**.*

*Note: warps aside, you are not constrained to scanning each block and then the grid row-wise like it's done here, but this is enough general-purpose that you rarely need another way to linearize thread indices.*

---

Available variables to each thread:
- `blockIdx(.x|.y|.z)`: idx of the thread's block
- `blockDim(.x|.y|.z)`: size of one block (threads per dimension)
- `threadIdx(.x|.y|.z)`: idx of the thread in its block
- `gridDim(.x|.y|.z)`: size of the grid of blocks (blocks per dimension)
- `warpSize`: number of threads per warp (implementation detail)

*Note: he type `dim3` is a 3D uint vector with default value 1 for unspecified dimensions.*

*Note: grid size = number of blocks, block size = number of threads per block*

---

Launching a kernel (2D example):
```
// you choose threads_per_row, threads_per_col
dim3 blockDim(threads_per_row, threads_per_col);
// given in_rows, in_cols for the data you need to process, infer the required grid
dim3 gridDim((in_rows + blockDim.x - 1)/blockDim.x, (in_cols + blockDim.y - 1)/blockDim.y);
// e.g.: one shared value per thread in the block
unsigned int sharedMemory = blockDim.x*blockDim.y*sizeof(...);
kernelFunction<<<numBlocks, blockSize, sharedMemory>>>(...);
cudaDeviceSynchronize();
```
All those "weird" sums before divisions are a fast way to compute the ceiling with integers:

Let $n, m \in \mathbb{N}$, then $\text{ceil}(n/m) = \text{floor}((n + m - 1)/m)$.

We can also write it as $\lceil \frac{n}{m} \rceil = \lfloor \frac{n + m - 1}{m} \rfloor$.

Then, recall that 'floor' is automatically applied by the division between unsigned integers in C/C++.
In truth, what happens is 'truncation', so for negative numbers it equates to applying 'ceil'.
But for index arithmetics this matters not, as it's almost always positive.
Therefore, by using the right side of the above equalities, we don't need to compute anything beside an integer sum and division, no ceil/floor required.

---

Available attributes:
- `__global__`: marks a function (kernel) callable from the host with the <<<...>>> syntax
- `__shared__`: marks memory that is shared by all threads of a block (vital for inter-thread **data re-use**)
  - `extern __shared__` the shared memory size is decided at kernel launch (last entry in <<<...>>>)
- `__device__`: marks a function or variable that lives on the device
- `__constant__`: marks constant memory, i.e., cached, read-only memory on the device
- `__host__`: marks a function as a host function (runs on the CPU), implied by default
- `__host__ __device__`: marks a function collable from both device and host (one version is compiled for each)
- `__managed__`: declares a variable in unified memory, accessible from both host and device, with automatic migration
- `__restrict__`: tells the compiler that a pointer does not alias with others, allowing better optimization (this is C++, not CUDA-specific)

---

The CUDA memory model:

<!--<img src="https://polimi365-my.sharepoint.com/:i:/r/personal/10669641_polimi_it/Documents/Images/CUDA_memory_model.PNG" alt="CUDA memory hierarchy overview." width="500" border="2">-->
<img src="https://i.imgur.com/yoEW1tU.png" alt="CUDA memory hierarchy overview." width="500" border="2">

*Credit: "Programming Massiveli Parallel Processors" by W. Hwu, D. Kirk, I. Hajj*

---

CUDA gives you small vector data types:
- `double2`
- `double3`
- `double4`
- `float2`
- `float3`
- `float4`
- `half`: 16bit float
- `half2`
- `bfloat16`: 8bit exponent (same as float), 7+1bit mantissa (low precision)
- `bfloat162`

---

Additional useful stuff:
- `__syncthreads()`: used inside a kernel to create a barrier (typical case: load data in shared memory -> sync -> compute on data)
- `int __syncthreads_count(predicate)`: sync and count how many instances of "predicate" are true among all threads
- `int __syncthreads_and(predicate)`, `int __syncthreads_or(predicate)`: same as above, but return the AND or OR of predicates
- `int __all(predicate)`: same as `__syncthreads_and`, but among threads in the same warp (used to manually speedup branches)
- `int __any(predicate)`: same as `__syncthreads_or`, but among threads in the same warp
- atomic operations: `atomicAdd`, `atomicSub`, `atomicMin`, `atomicMax`, `atomicInc` (increment), `atomicDec` (decrement), `atomicExch` (exchange), `atomicCAS` (compare-and-swap), `atomicAnd` (bitwise), `atomicOr`, `atomicXor`
- ` __threadfence_block()`: wait until all global and shared memory writes are visible to all threads in the same block
- ` __threadfence()`:  wait until all shared memory writes are visible to all threads in block and global memory writes are visible to all threads

Note: the CAS is very powerful to create global atomic locks!
```
// global variable: 0 unlocked, 1 locked
__device__ int lock = 0;
__global__ void kernel(...) {
  ...
  if (threadIdx.x == 0) {
    // acquire lock
    do {} while(atomicCAS(&lock, 0, 1));
    ...
     __threadfence();
    // release lock
    lock = 0;
  }
}
```

---

Warp shuffles:
- `T __shfl_up_sync(unsigned mask, T var, unsigned int delta)`, retrieves the value of `var` from another thread in the same warp that has warp_id `delta` lower than the caller
  - `mask` controls which threads are involved, usually set to -1 or 0xffffffff (every thread in the warp)
  - `var` is the local register variable from which to take the values to shuffle around
  - `delta` is the offset within the warp where to go fetch the value, if the appropriate thread does not exist (i.e. it's off the end of the warp) then the value is taken from the current thread
  - return value retrieved by the current thread
- `T __shfl_down_sync(unsigned mask, T var, unsigned int delta)`, same as above,but the `delta` is added to the caller's warp_id instead of subtracted
- `T __shfl_xor_sync(unsigned mask, T var, unsigned int lane_mask)` a XOR operation is performed between `lane_mask` and the calling thread's warp_id (or "lane") to determine the thread from which to copy the value (`lane_mask` controls which bits of the caller's id are "flipped")
- `__shfl_sync(unsigned mask, T var, unsigned int src_lane)`, simply go and fetch `var` from the thread with warp_id `src_lane`

---

Rules of thumb:
- the CUDA mentality is to launch exactly as many thread blocks as will fill
the GPU or match the input/output problem shape
- first fix the blockDim (# threads per block) then select gridDim (# of blocks) to fit the input (or output - depends on the algorithm's degree of parallelism)
- one can determine if a control construct can result in thread divergence by inspecting its decision condition. If the decision condition is based on threadIdx values, the control statement can potentially cause thread divergence
- unless you can prove no edge cases ever occur, every memory access needs to have a corresponding check that ensures that the indices used in the access are within the bounds of the array being accessed
- preload data in shared memory when more than one thread in a block accesses (reuses) the same element (ideally many times)
- imagine the mapping of global thread indices to data indices as a multivalued function, if the collective image of the threads in one block overlaps with the image of those in another, you will have duplicate elements in shared memory (this is fine, but you need more memory)
- you don't need to work with the idea of sub-cores and warp scheduling in mind, that's out of your control, but you need to keep in mind which threads end up in the same warp, and possibly which blocks (likely) in the same SM

---

Some general steps to compute control divergence:
- consider a single block, analyze how it is divided in warps
- identify blocks that have divergence
  - if the divergence is caused by a boundary condition, those are the blocks on the boundary
  - otherwise, you must find the underlying pattern causing the divergence
- for each of these blocks, identify the warps that have divergence
  - do not evaluate every block from scratch, bundle together those seeing the same divergence pattern
- you can now compute the fraction of diverged warps

<!--
Continuing:
- for each diverged warp, identify and count the distinct operations collectively required by its threads
- for non-diverged warp, follow once every possible independent path (control flow) they could follow, count its instructions
- for each diverged and not diverged control-flow you observe, count the warps that follow it
- the instructions overhead of divergece is given by the sum of the instructions needed by each warp, diverged and not, divided by the total instructions of non-diverged threads plus the average number of instructions between all paths times the number of diverged threads
-->

*Note: this notebook sometimes voluntarily changes coding conventions, like using 'i' for IDs instead of 'id'. This is intended to train the reader to understand someone else's code at a glance.* <!-- warning: retcon allert -->

*Note: much alike the above, many values that could be 'unsigned int' are written just as 'int' for readability. These kind of micro-optimizations are left to the reader.*

*Note: nvcc treats all code as C++, not C. Still, this notebook uses as little as possible syntax from C++.*

## **Exercise 1**

Test your CUDA environment (compiler support and available GPU card):

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1079b5db-aa98-6348-11c3-c91825cc7fa7)


In [ ]:
!nvidia-smi

Fri Sep 26 13:46:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

To know about the capabilities of you GPU, you can query the device properties and/or read the [compute capability specifications](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#features-and-technical-specifications__technical-specifications-per-compute-capability).<br>
Then, you should be able to answer the following questions:

* What are the maximum dimensions of a thread block?
* What is the maximum number of threads in a block?  
* What is the maximum size of the GPU global memory?
* What is the warp size?
* How many Streaming Multiprocessors are available?
* Is an execution configuration of <<<100, 1000>>> feasible on this device?
* Suppose you have a CUDA program using multiple blocks, should each block contain 4x4, 8x8, or 30x30 threads?
<!--
Universal answer: it depends.
In this case, depends on the size of the input, the amount of local variables used by each thread, and so on.
-->

In [ ]:
%%writefile deviceProperties.cpp
//                           ^^^ The extension should be 'cu', here we use 'cpp' to the Colab's syntax highlight working...
#include <stdio.h>

int main() {
  int deviceCount;
  cudaGetDeviceCount(&deviceCount);

  for (int dev = 0; dev < deviceCount; dev++){

    cudaDeviceProp deviceProp;
    cudaGetDeviceProperties(&deviceProp, dev);

    printf("Device properties of GPU card ID %d (%s)\n", dev, deviceProp.name);
    printf("  Compute capability: %d.%d\n", deviceProp.major, deviceProp.minor);
    printf("  Warp size: %d\n", deviceProp.warpSize);
    printf("  ------\n");
    printf("  Maximum GPU global memory size: %zu bytes\n", deviceProp.totalGlobalMem);
    printf("  Maximum GPU constants memory size: %zu bytes\n", deviceProp.totalConstMem);
    printf("  Maximum shared memory available per block: %zu bytes\n", deviceProp.sharedMemPerBlock);
    printf("  Maximum shared memory available per SM: %zu bytes\n", deviceProp.sharedMemPerMultiprocessor);
    printf("  ------\n");
    printf("  Maximum size of each dimension of a grid: %d x %d x %d\n", deviceProp.maxGridSize[0], deviceProp.maxGridSize[1], deviceProp.maxGridSize[2]);
    printf("  Maximum size of each dimension of a block: %d x %d x %d\n", deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1], deviceProp.maxThreadsDim[2]);
    printf("  Maximum number of threads per block: %d\n", deviceProp.maxThreadsPerBlock);
    printf("  Number of 32-bit registers available per block: %d\n", deviceProp.regsPerBlock);
    printf("  ------\n");
    printf("  Number of Streaming Multiprocessors: %d\n", deviceProp.multiProcessorCount);
    printf("  Maximum number of resident blocks per SM: %d\n", deviceProp.maxBlocksPerMultiProcessor);
    printf("  Maximum resident threads per SM: %d\n", deviceProp.maxThreadsPerMultiProcessor);
    printf("  Number of 32-bit registers available per SM: %d\n", deviceProp.regsPerMultiprocessor);

  }
}

Writing deviceProperties.cpp


In [ ]:
!mv deviceProperties.cpp deviceProperties.cu
!nvcc deviceProperties.cu -run

Device properties of GPU card ID 0 (Tesla T4)
  Compute capability: 7.5
  Warp size: 32
  ------
  Maximum GPU global memory size: 15828320256 bytes
  Maximum GPU constants memory size: 65536 bytes
  Maximum shared memory available per block: 49152 bytes
  Maximum shared memory available per SM: 65536 bytes
  ------
  Maximum size of each dimension of a grid: 2147483647 x 65535 x 65535
  Maximum size of each dimension of a block: 1024 x 1024 x 64
  Maximum number of threads per block: 1024
  Number of 32-bit registers available per block: 65536
  ------
  Number of Streaming Multiprocessors: 40
  Maximum number of resident blocks per SM: 16
  Maximum resident threads per SM: 1024
  Number of 32-bit registers available per SM: 65536


The **BEST** way to write a CUDA program is to:
- query the device for the available shared resource per block, warp size, etc.
- pick your block size and amount of shared memory accordingly

This way of doing things, however, gets quickly out of control in terms of added complexity.
So, this notebook will stick to plain and simple hardcoded examples.

## **Exercise 2**

Hardware questions, if a streaming multiprocessor (SM) has resource for:
- 32 threads per warp <!--(SIMD with 32 execution context at once)-->
- 2048 ($2^{11}$) threads
- 65536 ($2^{16}$) 32-bit registers
- at most 255 registers per thread

At maximum occupancy, how many registers does each execution context have? <!--this question is more like "what is an execution context", after you understand that it is a thread, the answer is clearly 32--><br>
Up to how many threads can execute the same instruction simultaneously (on different data)? <!--32, that's a warp-->

*Note: use more registers than are left in the SM and it will cause spilling of registers to local memory, significantly slowing threads down.*

---

These above are the specs for an A100, that has 108 SMs.

Suppose we have a kernel requesting 1000 blocks, each with 128 threads.
How does it get executed?

On current hardware, we would probably get 8-12 blocks running at the same time on each SM, and each block has 4 warps, resulting in 32-48 warps running on each SM.<br>
The goal is to use all SMs evenly, since only a few warps run at once per SM (one per sub-core).<br>
In any case, a SM can support at most 32 simultaneus blocks.

---

Assume now a device that allows up to 32 blocks per SM and 2048 threads per SM.
Indicate which of the following assignments per SM are possible.
In the cases in which it is possible, indicate the occupancy level.
1) 8 blocks with 128 threads each <!--yes, 8*128/2048=0.5 occupancy-->
2) 32 blocks with 32 threads each <!--yes, 32*32/2048=0.5 occupancy-->
3) 64 blocks with 32 threads each <!--no, would have been 64*32/2048=1.0 occupancy-->
4) 32 blocks with 64 threads each <!--yes, 64*32/2048=1.0 occupancy-->

The same device has 65536 ($2^{16}$) registers per SM. Assume that the kernel being run uses 128 registers per thread and no additional memory.
Indicate which of the following assignments per SM are possible.
In the cases in which it is possible, indicate the occupancy level.
1) 8 blocks with 128 threads each <!--no, requires 2^17 registers-->
2) 16 blocks with 32 threads each <!--yes, 16*32/2048=0.25 occupancy-->
3) 32 blocks with 32 threads each <!--no, requires 2^17 registers-->
4) 8 blocks with 64 threads each <!--yes, 8*64/2048=0.25 occupancy-->

*Note: underoccupancy can also arise from registers being oversubscribed. Registers are a fairly limited resource in modern GPUs!*

## **Exercise 3**

**Question 1:**

For a vector addition (element-wise sum), assume that the vector length is 8000, each thread calculates one output element, and the thread block size is 1024 threads. The programmer configures the kernel launch to have a minimal number of thread blocks to cover all output elements.

How many threads will be in the grid?

<!--
8 blocks, 8192 threads.
-->

**Question 2:**

We want to use each thread in a kernel to calculate two (adjacent) output elements of a vector addition. That is, we use half as many threads as vector elements.

What would be the expression for mapping the thread/block indices to data index for the two elements to be added?

<!--
data_idx_1 = (blockIdx.x * blockDim.x + threadIdx.x) * 2
data_idx_2 = (blockIdx.x * blockDim.x + threadIdx.x) * 2 + 1
-->

**Question 3:**

We want to use each thread in a kernel to calculate two output elements of a vector addition. Each thread block processes 2*blockDim.x consecutive elements that form two sections. All threads in each block will first process a section, each processing one element. They will then all move to the next section, again each processing one element. You can assume blockDim.x is a multiple of two.

What would be the expression for mapping the thread/block indices to data index for the two elements to be added?

If 'n' is the length of the vector and the thread block size is 1024 threads, what does the execution configuration ('<<<...>>>') look like?

<!--
data_idx_1 = blockIdx.x * blockDim.x * 2 + threadIdx.x
data_idx_2 = blockIdx.x * blockDim.x * 2 + threadIdx.x + blockDim.x / 2

<<ceil(N/(1024*2)), 1024>>
-->

**Question 4:**

A kernel is launched with '<<<(16, 16), (24, 12)>>>' on a device with warpSize = 32.

Write down the threadIdx.x and threadIdx.y at which the 2$^{nd}$ and 3$^{rd}$ warp of each block start and end.

<!--
Indices are given starting from 0.
warp 2: [2, 7] -> [4, 2]
warp 3: [4, 3] -> [6, 11]
warp 4: [7, 0] -> ...
-->

**Question 5:**

A kernel is launched with '<<<(16, 8, 16), (6, 4, 8)>>>' on a device with warpSize = 32.

Write down the threadIdx.x, threadIdx.y, and threadIdx.z at which the 1$^{st}$ and 2$^{nd}$ warp of each block start and end.

*Recall: threads are always assigned to warps in row-major order, with x first, then y, then z.*

<!--
Indices are given starting from 0.
warp 1: [0, 0, 0] -> [1, 1, 1]
warp 2: [2, 1, 1] -> [3, 2, 2]
warp 3: [4, 2, 2] -> ...
-->

<!--
Reasoning to find the index:
- where has left off the block before me?
- plus what I need to work on in my block.

Trick for blocks count:
- take the input size, add to it the number of inputs each block handles, minus 1, divide by the inputs per block to get a quick and dirty ceiling
-->

## **Exercise 4**

We are to process an 800x600 picture (800 pixels in width/x, 600 pixels in height/y) with the following `dumbPictureKernel()`.<br>
That is m = 600 and n = 800, with one thread per pixel.

Assume it is executed with blocks of 16x16 threads and warps of 32 threads:
1) How many warps will be generated during the execution of the kernel?
2) How many warps will have control divergence?
3) With m = 800 and n = 599, how many warps have control divergence?
4) With m = 799 and n = 600, how many warps have control divergence?

<!--
1) each block has 256 threads, that is exactly 8 warps; we need ceil(800/16)*ceil(600/16)*8 blocks, thus 1900*8 = 15200.
From another perspective, we issue 800x608 threads to cover the whole picture, exactly divided in warps of 32.
2) threads are packed in warps in row-major order (x-first). The width is exactly divisible by 16, so it never triggers the boundary condition in any block nor warp. The height has the last row of blocks that are on the boundary. The 8 warps of a block span 2 rows each. From point (1) we know that we use 608 rows of threads, of which only the first 600 are inside the boundary. Therefore, 8 rows of threads are outside the boundary, that is exactly 4 warps per block in the last row of blocks. Hence, no control divergence, only inactive warps.
3) Following from the answer to (2), there are now 9 rows of threads outside the boundary, resulting in 2 and 1/2 warps outside. Thus, exactly half of the threads in one warp diverge for every column of blocks, that is 50 diverged warps.
4) From the reasoning in (2), the height does not cause any divergence, either a warp is entirely inside or outside the boundary. Along x, now, the last column of warps all diverge, and with a warp being 2 rows tall, that is 300 warps diverging for 2 threads out of 32. It is not 304 because the last 4 warps (or 8 rows) are in any case entirely outside the height's boundary.

Note: 'n' is the bound along columns, and columns are along x.
-->

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void dumbPictureKernel(float* d_Pin, float* d_Pout, int n, int m) {
  // calculate the row idx of the d_Pin and d_Pout element to process
  int row = blockIdx.y*blockDim.y + threadIdx.y;
  // calculate the column idx of the d_Pin and d_Pout element to process
  int col = blockIdx.x*blockDim.x + threadIdx.x;

  // each thread computes one element of d_Pout if in range
  if ((row < m) && (col < n)) {
    d_Pout[row*n + col] = 2*d_Pin[row*n + col];
  }
}

**This example is more common than you may think:**

A typical reason for using a control construct with thread divergence is handling boundary conditions when mapping threads to data.
This is because the total number of threads needs to be a multiple of the block size whereas the size of the data can be an arbitrary number.

Fortunately, as the input size grows (1D - linearly, 2D - quadratically, 3D - cubically), the boundary becomes smaller (1D - no growth, 2D - linearly, 3D - quadratically).
Think how the perimeter of a square is always less than its area, even moreso when the sidelength grows.
This means that, as we scale up, divergence affects a smaller fraction of warps, resulting in a lower overhead percentage.

## **Exercise 5**

Given the following vector addition program, estimate what impact will have the serialization due to control divergence (aka warp divergence). Assume warps of 32 threads.

Quantify:
- the percentage of warps encoutering divergence
- the SIMD efficiency for the addition operation in the kernel (# threads doing it in parallel / # threads not doing it in parallel) in each diverged warp

<!--
We have 4 blocks of 256 threads, 1024 total threads for 1000 elements of input, so the boundary condition triggers only for those last 24 threads = 1 warp.
Since 32 divides 256 exactly, in total we have 4*256/32 warps, the divergence percentage is 1/32.
In that sole diverged warp, 8 out of 32 threads will run the sum, others will be idle, so the SIMD efficiency is 8/32.
-->

In [ ]:
%%writefile vecadd2.cpp
#include<stdio.h>

__global__
void vecAddKernel(float* A, float* B, float* C, int n) {
  int i = blockDim.x*blockIdx.x + threadIdx.x;
  if (i < n)
    C[i] = A[i] + B[i];
}

int main() {
  int n = 1000;

  // timing
  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, n*sizeof(float));
  cudaMallocManaged(&b, n*sizeof(float));
  cudaMallocManaged(&c, n*sizeof(float));

  // arbitrary initialization
  for (float i = 0; i < n; i++) {
    a[i] = i;
    b[i] = i + 5;
  }

  cudaEventRecord(start);
  vecAddKernel<<<4, 256>>>(a, b, c, n);
  cudaEventSynchronize(stop);
  cudaEventRecord(stop);
  cudaEventElapsedTime(&milli, start, stop);

  // unfortunately, this won't pick up on the slight overhead of divergence...
  printf("Execution time (ms): %f \n", milli);

  cudaEventDestroy(stop);
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

Writing vecadd2.cpp


In [ ]:
!mv vecadd2.cpp vecadd2.cu
!nvcc -arch=sm_75 vecadd2.cu -run

Execution time (ms): 0.000000 


How divergence occurs:
- small branch (like above): all threads execute both branches with masking
- big branches (~10+ instructions each): the compiler inserts "voting instructions" to speedup the case where all threads go in the same branch, if that does not happen, the cost is again the sum of both branches

Branches are handled per-warp, so the maximum throughput penalty is 1/32x on a A100 if ony one thread goes down a different path in every warp.

Workarounds:
- determine at compile time that all threads in the warp must go the same way (thank you very much)
- if you can know a priori which part of the input takes which branch, create two or more kernels, each executing one branch, and launch them on their respective inputs

Example:<br>
Processing a long list of elements where, depending on run-time values, a few require very expensive processing.<br>
GPU implementation:<br>
First process the list to build two sub-lists of "simple" and "expensive" elements, then process the two sub-lists separately. You need 3 kernels, one per element type and one to filter the list.

*Note: none of this is new, this is what we did 35 years ago on CRAY machines.*

## **Exercise 6**

Given a vector, double the value of its elements in even position, while put to zero the elements in odd position if they are less than 1.0, otherwise leave them unchanged.

This kernel implements the optimization proposed in Exercise 5.
Therefore it replaces a divergence-inducing branch with two kernels, one operating on odd and one on even position elements.

Questions:
1) what would have been the impact of control divergence if we had only a single kernel doing both tasks with a `if` checking if the current index was odd/even? Assume warps of 32 threads.
<!--
Every single warp would have diverged.
-->
2) could we do the same "split the kernel in one per branch" trick to avoid divergence due to the `if(v[i] < 1.0)` check?
<!--
No, the trick is applicable only on boundary conditions, that is checks strictly dependent on the thread's id/position in the grid, block, or warp.
-->

In [ ]:
%%writefile vecadd2.cpp
#include<stdio.h>

__global__
void evenElements(float* v, int n) {
  int i = blockDim.x*2*blockIdx.x + threadIdx.x*2;
  if (i < n)
    v[i] = v[i]*2;
}

__global__
void oddElements(float* v, int n) {
  int i = blockDim.x*2*blockIdx.x + threadIdx.x*2 + 1;
  if (i < n)
    if (v[i] < 1.0)
      v[i] = 0.0;
}

int main() {
  // HP: let n always be even
  int n = 1000;
  int numThreads = (n + 1)/2;
  int blockSize = 32;
  int numBlocks = (numThreads + blockSize - 1)/blockSize;

  float *v;
  cudaMallocManaged(&v, n*sizeof(float));
  for (float i = 0; i < n; i++)
    v[i] = i/10;

  evenElements<<<numBlocks, blockSize>>>(v, n);
  oddElements<<<numBlocks, blockSize>>>(v, n);
  cudaDeviceSynchronize();

  cudaFree(v);
  return 0;
}

In [ ]:
!mv vecadd2.cpp vecadd2.cu
!nvcc -arch=sm_75 vecadd2.cu -run

A big catch:

Since per-thread work is extremely tiny, in this case the divergence penalty is very small. On the other hand, doing two global sweeps, complete with kernel launch and all, adds enough overhead to dominate over the benefit of not diverging.

While this trick of creating multiple kernel is useful, reserve it for when the amount of work along either branch is quite substantial.

*Note: no need to synchronize between kernel calls, all CUDA activity issued to a single device will not overlap in any way (unless using CUDA streams).*

## **Exercise 7**

Given the following two kernels, estimate what impact will have the serialization due to control divergence (assume warps of 32 threads).

In both kernels, we assign to an output element the maximum that can be found between its matching input position an a certain number of positions ahead.

<!--
1D case:
We have 4 blocks each with 256 threads.
All threads run the loop for the same number of iterations, except the last 16 of the very last block.
So we have a 16/1024 diverged threads.
Note however that each thread diverges by a different amount, the very last diverges for all iterations ("wasting" them), while the second-to-last only for one less iteration, and so on.
A warp is 32 threads, and since 32 divides exactly the number of threads per block (256), diverged threads will be together in the last warp.
Hence, we have 1/(4*(256/32)) divergence ration.

3D case:
We have 4*4*4 blocks each of 256*256*256 threads.
Here, each thread looks at a "cube" DxDxD ahead of itself in all directions.
We have divergence whenever a thread has its "cube" go out of the input tensor's boundary.
Hence we will have a box of DxNxN threads diverging because of x, another identical box of diverging threads because of y, and so for z. Those boxed however overlap pair-wise and also all 3 in a corner, so we have actually 3*(D*N*N) - 3*(D*D*N) + (D*D*D) = 49549312 diverged threads.
For warps, we have 256^3/32 of them per block, all unfolding along the x dimension (row-major), and since they divide blockDim.x exactly, no warp "wraps back" to a different y or z. This means that each warp is a thin box "32x1x1" and 4 of them on top of each other span the whole width of a block.
So, 1xNxN warps will diverge because of x, N/32xDxN because of y, and N/32xNxD because of z, those boxes again overlap, so the final count is: 1*N*N + N/32*D*N + N/32*N*D - 1*D*N - 1*N*D - N/32*D*D + 1*D*D = 2056448 diverged warp out of N*N*N/32.
-->

In [ ]:
%%writefile only_for_highlight.cpp

#define N = 1024
#define D = 16

__global__ void maxInNextElements(float* v, float* o, int n) {
  int g_id = blockIdx.x*blockDim.x + threadIdx.x;
  int limit = min(D, gridDim.x*blockDim.x - g_id);

  float res = 0.0;
  for (int i = 0; i < limit; i += 1) {
    res = max(res, v[g_id + i]);
  }
  o[g_id] = res;
}

// launched with:
int main() {
  float v[N], o[N];
  //...
  maxInNextElements<<<4, N/4>>>(v, o, N);
  //...
}

In [ ]:
%%writefile only_for_highlight.cpp

#define N = 1024
#define D = 16

__global__ void maxInNextElements(float* v, float* o, int n) {
  int g_id_x = blockIdx.x*blockDim.x + threadIdx.x;
  int g_id_y = blockIdx.y*blockDim.y + threadIdx.y;
  int g_id_z = blockIdx.z*blockDim.z + threadIdx.z;
  int limit_x = min(D, gridDim.x*blockDim.x - g_id_x);
  int limit_y = min(D, gridDim.y*blockDim.y - g_id_y);
  int limit_z = min(D, gridDim.z*blockDim.z - g_id_z);

  float res = 0.0;
  // this looks "a cube" ahead
  for (int i = 0; i < limit_x; i += 1) {
    int x = g_id_x + i;
    for (int j = 0; j < limit_y; j += 1) {
      int y = g_id_y + j;
      for (int k = 0; k < limit_z; k += 1) {
        int z = g_id_z + k;
        res = max(res, v[x + y*N + z*N*N]);
      }
    }
  }
  o[g_id_x + g_id_y*N + g_id_z*N*N] = res;
}

// launched with:
int main() {
  float v[N*N*N], o[N*N*N];
  //...
  maxInNextElements<<<(4, 4, 4), (N/4, N/4, N/4)>>>(v, o, N);
  //...
}

## **Exercise 8**

The following kernel takes a vector and updates each element as the average of itself, its two following neighbors, and its two preceeding neighbors.
If an element is on the boundary of the vector, outside elements are considered to be 0.

Crucially, the kernel is sped-up by loading data in shared memory for reuse among threads in the same block.
The canonical term to refer to reuse in this case is "halo", because if you imagine to run the kernel sequentially over the vector, left to right, as you move you would be continously reusing 4-out-of-5 elements that you just used for the immediately prior computation, that is, your input at each vector position continously overlaps with the "lingering shadow" of that of the previous one.

Be wary that the shared memory allocation needs to be slightly larger than the number of elements handled by the block.
It needs to store 4 extra values, 2 below and 2 above the current block.

In [ ]:
%%writefile only_for_highlight.cpp
#include<stdio.h>

__global__
void neighborsAverage(const float* __restrict__ in, float* __restrict__ out, int n) {
  extern __shared__ float s[];
  int t_id = threadIdx.x;
  int g_t_id = blockIdx.x*blockDim.x + t_id;

  // load into shared memory with halo of 2 on each side
  int l_idx = t_id + 2; // make space for the first two elements
  if (g_t_id < n)
    s[l_idx] = in[g_t_id];
  else
    s[l_idx] = 0.0f;

  // load left halo
  if (t_id < 2) {
    int left_idx = g_t_id - 2 + t_id;
    s[t_id] = (left_idx >= 0) ? in[left_idx] : 0.0f;
  }
  // load right halo
  if (t_id >= blockDim.x - 2) {
    int right_idx = g_t_id + (t_id - (blockDim.x - 2)) + 1;
    s[l_idx + 2] = (right_idx < n) ? in[right_idx] : 0.0f;
  }

  __syncthreads();

  // 5 elements average
  if (g_t_id < n) {
    out[g_t_id] = (s[l_idx-2] + s[l_idx-1] + s[l_idx] + s[l_idx+1] + s[l_idx+2]) / 5.0f;
  }
}

int main() {
  const int n = 4096;
  float h_in[n], h_out[n];
  for (int i = 0; i < n; i++)
    h_in[i] = i + 1;

  float *d_in, *d_out;
  cudaMalloc(&d_in, n*sizeof(float));
  cudaMalloc(&d_out, n*sizeof(float));
  cudaMemcpy(d_in, h_in, n*sizeof(float), cudaMemcpyHostToDevice);

  int blockSize = 256;
  int gridSize = (n + blockSize - 1)/blockSize;
  size_t shmem = (blockSize + 4) * sizeof(float); // +4 for halos
  neighborsAverage<<<gridSize, blockSize, shmem>>>(d_in, d_out, n);
  cudaMemcpy(h_out, d_out, n*sizeof(float), cudaMemcpyDeviceToHost);

  for (int i = 0; i < n; i++)
    printf("%6.2f ", h_out[i]);
  printf("\n");

  cudaFree(d_in);
  cudaFree(d_out);
  return 0;
}


If, as shown, we launch the kernel with blocks of 256 threads, and the vector has length 4096, what is the impact of control divergence? Assume 32 threads per warp.

<!--
We have 32 blocks, the division is exact, the 'g_t_id < n' condition does not cause divergence.
In each block, the first two and last two threads take the "left halo" and "right halo" branches respectively, doing an extra shared memory load.
So we have 4 diverged threads per block: 4*32/4096 thread divergence ratio.
These threads end up in the first and last warp of each block.
Each block has 256/32 = 8 warps, therefore we have a 2/8 warp divergence ratio.
Moreover, each warp diverges by 2/32 threads.
-->

*Note: here divergence arises from the extra loads in shared memory!*

## **Exercise 9**

Consider the following code snippet, representing a (sub-optimal) matrix multiplication kernel:

In [ ]:
%%writefile only_for_highlight.cpp

// HP: square matrices of side length 'n'
__global__
void MatrixMulKernel(float* A, float* B, float* C, int n) {
  int row = blockIdx.y*blockDim.y + threadIdx.y;
  int col = blockIdx.x*blockDim.x + threadIdx.x;
  if ((row < n) && (col < n)) {
    float partial_out = 0;
    for (int k = 0; k < n; ++k) {
      partial_out += A[row*n + k] * B[k*n + col];
    }
    C[row*n + col] = partial_out;
  }
}

If we launch the kernel with a block size of 16x16 on 1000x1000 matrices, how many warps will have control divergence?
Assume warpSize = 64 (fictional, but you never know...).

<!--
The boundary condition is on the global 'row' and 'col' indices.
ceil(1000/16) = 63 blocks, 63*16 = 1008 threads along both x and y.
In each block we have 4 warps, each spanning 4 rows.
Along y, only the last 8 rows are outside the boundary, that means 2 warps per column are fully outside, no divergence.
Along x, the last column of blocks is vertically cut in half inside and half ouside the boundary, thus every warp is subject to divergence except warps that are also entirely outside along y too. With one warp every 4 rows, there are 252 warps in the last column of blocks, the last 2 are ouside the y boundary, so 250 warps will diverge.
-->

With an SM that can support up to 1,536 threads and up to 8 blocks, which of the following block configuration would result in the most number of threads in each SM: 64, 128, 512 or 1024 threads per block?

<!--
1536/64 = 24, not good, > 8
1536/128 = 12, not good, > 8
1536/512 = 3, the only viable option to use all threads
1536/1024 = 1.5, not good, does not divide threads exactly (blocks are either assigned entirely or not assigned to a SM)
-->

A little visual help:

<!--<img src="https://polimi365-my.sharepoint.com/:i:/r/personal/10669641_polimi_it/Documents/Images/blocks_over_input.png" alt="2D visualization of the input, blocks, and warps." width="500" border="2">-->
<img src="https://i.imgur.com/fNjRp2u.png" alt="2D visualization of the input, blocks, and warps." width="500" border="2">

*Note: ops, "input" should have been "output", well, too late for that.*

## **Exercise 10**

**Question 1:**

Assume that each atomic operation in a DRAM system has a total latency of 100ns.
What is the maximal throughput we can get for atomic operations on the same global memory variable?

<!--
Explanation: No other atomic operation can touch the same variable for the entire duration of 100ns.
The maximal rate is 1/100ns = 0.01G atomic operations per second.
-->

Assuming now that the kernel does <!--(total over all threads)--> 5 floating-point operations per atomic operation, what is the maximal floating-point throughput of the kernel execution as limited by the throughput of the atomic operations?

<!--
The maximal is 5 operations every 100ns, 5/100ns = 0.05 GFLOPS.
-->

**Question 2:**

For a GPU that supports atomic operations in L2 cache, assume that each atomic operation takes 4ns to complete in L2 cache and 100ns to complete in DRAM.
Assume that 90% of the atomic operations hit in L2 cache. What is the approximate throughput for atomic operations on the same global memory variable?

<!--
The average latency is 4ns * 90% + 100ns * 10% = 13.6ns.
The average throughput is approximately 1/13.6ns = 0.0735G atomic operations per second.
-->

**Question 3:**

Continuing from Question 1.

Assume that we privatized some global memory variables into shared memory variables, and the shared memory access latency is 1ns.
For simplicity, assume that the additional global memory atomic operations for accumulating privatized variable into the global variable adds 10% to the total execution time.
Assume that a kernel performs 5 floating-point operations per atomic operation. What is the maximal floating-point throughput of the kernel execution as limited by the throughput of the atomic operations?

<!--
The effective throughput without the final accumulation to the global variable is 5/1ns = 5 GFLOPS.
Since the time is stretched by 10%, the final effective throughput is approximately 5/1.1 = 4.5 GFLOPS.
-->

## **Exercise 11**

Consider the following basic reduction kernel code fragment:

In [ ]:
%%writefile only_for_highlight.cpp

int t = threadIdx.x;
int start = 2*blockDim.x*blockIdx.x; // each block of threads starts with twice the elements as there are threads, that is to have one sum per thread in the first reduction round/level
extern __shared__ float partial_sum[];
partial_sum[t] = input[start + t]; // each thread sets up two values, ultimately all values are preloaded in 'partial_sum'
partial_sum[blockDim.x + t] = input[start + blockDim.x + t];

for (int stride = 1; stride <= blockDim.x; stride *= 2) {
  __syncthreads();
  if (t % stride == 0) // one thread every 'stride' sums its corresponding value and the next one from the previous round
    partial_sum[2*t] += partial_sum[2*t + stride];
}

Let the block size be 1024 and the warp size be 32.
Assume the length of 'input' to always be a multiple of the block size.

Questions:
1) how many warps in a block will have divergence during the iteration where stride = 1?
<!--
With stride = 1, the condition 't % stride == 0' is always true, no divergence occurs.
From another perspective, the kernel is set up such that during the first iteration each thread executes exactly one of the partial sums in the first reduction level.
-->
2) how many warps in a block will have divergence during the iteration where stride is equal to 16?
<!--
This is the fifth iteration/reduction level.
Only one in sixteen threads will enter the 'if' and perform the sum.
For how the reduction is performed that is exactly one thread every sixteen going by thread id, therefore every warp will have two threads taking the branch and 30 not.
Every warp diverges.
-->
3) how many warps in a block will have divergence during the iteration where stride is equal to 64?
<!--
This is the seventh iteration/reduction level.
For the same reasoning as (2), one in 64 threads will take the branch, that is one thread every two warps.
Hence, half of the warps diverge.
-->

## **Exercise 12**

Considering the following improved reduction kernel:

In [ ]:
%%writefile only_for_highlight.cpp

int t = threadIdx.x;
int start = 2*blockDim.x*blockIdx.x;
extern __shared__ float partial_sum[];
partial_sum[t] = input[start + t]; // omitting the boundary check...
partial_sum[blockDim.x + t] = input[start + blockDim.x + t];

for (int stride = blockDim.x; stride > 0; stride /= 2) { // be wary, here we halve the stride at every iteration, unlike in the previous exercise!
  __syncthreads();
  if (t < stride) // 'stride' threads sums their corresponding values with those one 'stride' ahead
    partial_sum[t] += partial_sum[t + stride];
}

Let the block size be 1024 and the warp size be 32.
Assume the length of 'input' to always be a multiple of the block size.

1) how many warps in a block will have divergence during the iteration where stride is equal to 16?
<!--
This is the seventh iteration.
For how this version is written (focus on the boundary condition), only the first 'stride' threads in a block take the branch.
Therefore, only the first warp in a block will diverge, while also being the sole warp doing useful work.
-->
2) how many warps in a block will have divergence during the iteration where stride is equal to 64?
<!--
This is the fifth iteration.
Following from (2), only the first 64 threads per block are active.
Therefore, the first two warps in a block will fully take the branch, while the others don't do any useful work.
No warp diverges.
-->

## **Exercise 13**

Consider the work efficient scan kernel based on reduction trees and inverse reduction trees, and focus on the following code fragment:

In [ ]:
%%writefile only_for_highlight.cpp

extern __shared__ float scan_sum[];
scan_sum[threadIdx.x*2] = input[blockDim.x*blockIdx.x*2 + threadIdx.x*2]; // omitting the boundary checks...
scan_sum[threadIdx.x*2 + 1] = input[blockDim.x*blockIdx.x*2 + threadIdx.x*2 + 1];
__syncthreads();

// === FOCUS ON THIS ===
// == REDUCTION PHASE ==
for (int stride = 1; stride < blockDim.x; stride *= 2) {
  int index = (threadIdx.x + 1)*stride*2 - 1;
  if (index < 2*blockDim.x)
    scan_sum[index] += scan_sum[index - stride];
  __syncthreads();
}
// =====================

for (int stride = blockDim.x/2; stride > 0; stride /= 2) {
  __syncthreads();
  int index = (threadIdx.x + 1)*stride*2 - 1;
  if (index + stride < 2*blockDim.x)
    scan_sum[index + stride] += scan_sum[index];
}
__syncthreads();
result[i] = scan_sum[threadIdx.x]; // omitting the boundary check...

Assume that we give 2048 elements to each block. Each block thus has 1024 threads handling that section of elements.
Let warp size be 32.

How many warps in each block will have control divergence during the reduction tree phase iteration where stride is 16?

<!--
The stride doubles at every iteration, along with each thread's 'index'.
When stride is 16, the first thread in a block will have index = 31, the second index = 63, and so on until the 32nd (threadIdx.x == 31) thread will have index = 1023.
After the 32nd thread, all others will not take the branch.
Therefore, not warp diverges, the first warp will do useful work, all others won't take the branch.
-->

## **Exercise 14**

Consider the work inefficient scan kernel based on interleaved reduction trees, and focus on the following code fragment:

In [ ]:
%%writefile only_for_highlight.cpp

extern __shared__ float scan_sum[];
scan_sum[threadIdx.x] = input[blockDim.x*blockIdx.x + threadIdx.x]; // omitting the boundary checks...
__syncthreads();

for (int stride = 1; stride < blockDim.x; stride = stride*2) {
    __syncthreads();
    float tmp = threadIdx.x >= stride ? scan_sum[threadIdx.x - stride] : 0.0;
    __syncthreads();
    scan_sum[threadIdx.x] += tmp;
}

Assume that we have 1024 elements in each block and warp size is 32, how many warps per block will have control divergence during the iteration where stride is 16?
<!--
The stride doubles at every iteration, each time deactivating an additional half of the threads.
When stride is 16, all threads but the first 16 are doing active work. Therefore only the first warp per block diverges exactly by half of its threads.
-->
How many warps in each block will have control divergence during the iteration where stride is 64?
<!--
From the above, when stride is 64 the first 64 threads will be inactive. This amounts to the entirety of the first two warps, that will collectively take the inactive branch without diverging.
-->

## **Host-Device Data Transfers**

Let's use a profiler to analyze the performance of three different ways to transfer data from the host to the device:
* Manual memory allocation
* Unified Memory
* Asynchronous memory prefetching
* Pinned Memory

For a deeper discussion, see https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/#memory-optimizations.

### Manual Memory Allocation (explicit `cudaMemcpy`)

This is our classic baseline with explicit data transfers between host and device.

In [ ]:
%%writefile vecadd_manual.cpp
#include <stdio.h>
#include <stdlib.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

// use '__device__' instead if the value is meant to be modifiable by the kernel
__constant__ float extra[1];

__global__
void vectorAdd(float* add1, float* add2, float* result, int N) {
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = id; i < N; i+= stride)
    result[i] = add1[i] + add2[i] + *extra;
}

void initWith(float number, float* vec, int size) {
  for (int i = 0; i < size; i++)
    vec[i] = number;
}

void checkResult(float number, float* vec, int size) {
  for (int i = 0; i < size; i++) {
    if (vec[i] != number) {
      printf("Error at %d: %f != %f\n", i, vec[i], number);
      exit(1);
    }
  }
}

int main() {
  const int N = 1 << 24;
  int blockSize = 1024;
  int numBlocks = (N + blockSize - 1) / blockSize;

  // host side arrays
  float *h_a;
  float *h_b;
  float *h_c;
  float host_extra = 12.0f;

  h_a = (float*)malloc(N*sizeof(float));
  h_b = (float*)malloc(N*sizeof(float));
  h_c = (float*)malloc(N*sizeof(float));

  initWith(3.0f, h_a, N);
  initWith(4.0f, h_b, N);
  initWith(0.0f, h_c, N);

  // device side arrays
  float *d_a;
  float *d_b;
  float *d_c;

  cudaMalloc((void**)&d_a, N*sizeof(float));
  cudaMalloc((void**)&d_b, N*sizeof(float));
  cudaMalloc((void**)&d_c, N*sizeof(float));

  // manual copy
  cudaMemcpy(d_a, h_a, N*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b, N*sizeof(float), cudaMemcpyHostToDevice);
  gpuErrchk(cudaMemcpyToSymbol(extra, &host_extra, sizeof(float), 0, cudaMemcpyHostToDevice));

  // kernel
  vectorAdd<<<numBlocks, blockSize>>>(d_a, d_b, d_c, N);

  //gpuErrchk(cudaPeekAtLastError());
  gpuErrchk(cudaDeviceSynchronize());

  cudaMemcpy(h_c, d_c, N*sizeof(float), cudaMemcpyDeviceToHost);

  checkResult(7.0f + 12.0f, h_c, N);
  printf("Manual memory allocation: success!\n");

  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  free(h_a);
  free(h_b);
  free(h_c);

  return 0;
}

Writing vecadd_manual.cpp


### 2. Unified Memory (`cudaMallocManaged`)

No explicit copies needed, the GPU and CPU share the same allocation.

In [ ]:
%%writefile vecadd_unified.cpp
#include <stdio.h>
#include <stdlib.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

__global__
void vectorAdd(float* add1, float* add2, float* result, int N) {
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = id; i < N; i += stride)
    result[i] = add1[i] + add2[i];
}

void initWith(float number, float* vec, int size) {
  for (int i = 0; i < size; i++)
    vec[i] = number;
}

void checkResult(float number, float* vec, int size) {
  for (int i = 0; i < size; i++) {
    if (vec[i] != number) {
      printf("Error at %d: %f != %f\n", i, vec[i], number);
      exit(1);
    }
  }
}

int main() {
  const int N = 1 << 24;
  const int blockSize = 1024;
  const int numBlocks = (N + blockSize - 1) / blockSize;

  float *a, *b, *c;
  cudaMallocManaged(&a, N * sizeof(float));
  cudaMallocManaged(&b, N * sizeof(float));
  cudaMallocManaged(&c, N * sizeof(float));

  initWith(3.0f, a, N);
  initWith(4.0f, b, N);
  initWith(0.0f, c, N);

  vectorAdd<<<numBlocks, blockSize>>>(a, b, c, N);

  //gpuErrchk(cudaPeekAtLastError());
  gpuErrchk(cudaDeviceSynchronize());

  checkResult(7.0f, c, N);
  printf("Unified Memory: success!\n");

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}

Writing vecadd_unified.cpp


### Unified Memory with Asynchronous Prefetching

An improvement over the "unified memory" case.<br>
Here we tell CUDA in advance to move the memory to the GPU, which avoids stalls during the kernel.

In [ ]:
%%writefile vecadd_prefetch.cpp
#include <stdio.h>
#include <stdlib.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

__global__
void vectorAdd(float* add1, float* add2, float* result, int N) {
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = id; i < N; i += stride)
    result[i] = add1[i] + add2[i];
}

void initWith(float number, float* vec, int size) {
  for (int i = 0; i < size; i++)
    vec[i] = number;
}

void checkResult(float number, float* vec, int size) {
  for (int i = 0; i < size; i++) {
    if (vec[i] != number) {
      printf("Error at %d: %f != %f\n", i, vec[i], number);
      exit(1);
    }
  }
}

int main() {
  const int N = 1 << 24;
  const int blockSize = 1024;
  const int numBlocks = (N + blockSize - 1) / blockSize;

  float *a, *b, *c;
  cudaMallocManaged(&a, N * sizeof(float));
  cudaMallocManaged(&b, N * sizeof(float));
  cudaMallocManaged(&c, N * sizeof(float));

  initWith(3.0f, a, N);
  initWith(4.0f, b, N);
  initWith(0.0f, c, N);

  int device;
  cudaGetDevice(&device);

  // explicitly migrate data before the kernel
  cudaMemPrefetchAsync(a, N * sizeof(float), device);
  cudaMemPrefetchAsync(b, N * sizeof(float), device);

  vectorAdd<<<numBlocks, blockSize>>>(a, b, c, N);

  //gpuErrchk(cudaPeekAtLastError());
  gpuErrchk(cudaDeviceSynchronize());

  // prefetch back to host for validation
  cudaMemPrefetchAsync(c, N * sizeof(float), cudaCpuDeviceId);
  gpuErrchk(cudaDeviceSynchronize());

  checkResult(7.0f, c, N);
  printf("Unified Memory + Prefetching: success!\n");

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}


Writing vecadd_prefetch.cpp


### Pinned Memory

Pinned (or page-locked) host memory sits between the "manual copy" and "unified memory" approaches in complexity and performance.<br>
It's still manually managed, but it allows asynchronous and faster host-device transfers, because the memory can be accessed directly by the GPU's DMA engine without an extra staging step.

*Note: pinned memory should be the fastest manual transfer method, especially when overlapping transfers and computation using streams. May still be inferior to the asynch method depending on the nature and frequency of the transfers.*

*Note: pinned memory should not be overused. Excessive use can reduce overall system performance because pinned memory is a scarce resource, but how much is too much is difficult to know in advance.*

In [ ]:
%%writefile vecadd_pinned.cpp
#include <stdio.h>
#include <stdlib.h>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true) {
   if (code != cudaSuccess) {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort)
        exit(code);
   }
}

__global__
void vectorAdd(float* a, float* b, float* c, int N) {
  int id = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = id; i < N; i += stride)
    c[i] = a[i] + b[i];
}

void initWith(float number, float* vec, int size) {
  for (int i = 0; i < size; i++)
    vec[i] = number;
}

void checkResult(float number, float* vec, int size) {
  for (int i = 0; i < size; i++) {
    if (vec[i] != number) {
      printf("Error at %d: %f != %f\n", i, vec[i], number);
      exit(1);
    }
  }
}

int main() {
  const int N = 1 << 24;
  const int blockSize = 256;
  const int numBlocks = (N + blockSize - 1) / blockSize;

  // allocate pinned memory
  float *h_a, *h_b, *h_c;
  gpuErrchk(cudaMallocHost((void**)&h_a, N * sizeof(float)));
  gpuErrchk(cudaMallocHost((void**)&h_b, N * sizeof(float)));
  gpuErrchk(cudaMallocHost((void**)&h_c, N * sizeof(float)));

  initWith(3.0f, h_a, N);
  initWith(4.0f, h_b, N);
  initWith(0.0f, h_c, N);

  // device arrays
  float *d_a, *d_b, *d_c;
  gpuErrchk(cudaMalloc((void**)&d_a, N * sizeof(float)));
  gpuErrchk(cudaMalloc((void**)&d_b, N * sizeof(float)));
  gpuErrchk(cudaMalloc((void**)&d_c, N * sizeof(float)));

  // asynchronous memory copy using pinned host memory
  cudaStream_t stream;
  gpuErrchk(cudaStreamCreate(&stream));

  gpuErrchk(cudaMemcpyAsync(d_a, h_a, N * sizeof(float), cudaMemcpyHostToDevice, stream));
  gpuErrchk(cudaMemcpyAsync(d_b, h_b, N * sizeof(float), cudaMemcpyHostToDevice, stream));

  vectorAdd<<<numBlocks, blockSize, 0, stream>>>(d_a, d_b, d_c, N);
  gpuErrchk(cudaGetLastError());

  gpuErrchk(cudaMemcpyAsync(h_c, d_c, N * sizeof(float), cudaMemcpyDeviceToHost, stream));

  gpuErrchk(cudaStreamSynchronize(stream));
  gpuErrchk(cudaStreamDestroy(stream));

  checkResult(7.0f, h_c, N);
  printf("Pinned memory: success!\n");

  // cleanup
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  cudaFreeHost(h_a);
  cudaFreeHost(h_b);
  cudaFreeHost(h_c);

  return 0;
}


Writing vecadd_pinned.cpp


### Run!

In [ ]:
!mv vecadd_manual.cpp vecadd_manual.cu
!mv vecadd_unified.cpp vecadd_unified.cu
!mv vecadd_prefetch.cpp vecadd_prefetch.cu
!mv vecadd_pinned.cpp vecadd_pinned.cu
print("\nCompiling & running:")
!nvcc -arch=sm_75 vecadd_manual.cu -o vecadd_manual -run
!nvcc -arch=sm_75 vecadd_unified.cu -o vecadd_unified -run
!nvcc -arch=sm_75 vecadd_prefetch.cu -o vecadd_prefetch -run
!nvcc -arch=sm_75 vecadd_pinned.cu -o vecadd_pinned -run


Compiling & running:
Manual memory allocation: success!
Unified Memory: success!
Unified Memory + Prefetching: success!
Pinned memory: success!


Then, the most indicative metrics are under `cuda_gpu_mem_time_sum`.<br>
Additionally, under `cuda_api_sum`, look out for:
- `cudaMalloc` + `cudaMemcpy`
- `cudaMallocManaged`
- `cudaMallocManaged` + `cudaMemPrefetchAsync`

Either look for their combined total time or combined percentage of time.<br>
In theory, it should reduce as we go down the list.

In [ ]:
print("\n============= PROFILE: vecadd_manual =============\n")
!nsys profile --stats=true ./vecadd_manual
print("\n============= PROFILE: vecadd_unified ============\n")
!nsys profile --stats=true ./vecadd_unified
print("\n============ PROFILE: vecadd_prefetch ============\n")
!nsys profile --stats=true ./vecadd_prefetch
print("\n============ PROFILE: vecadd_pinned ============\n")
!nsys profile --stats=true ./vecadd_pinned


============= PROFILE: vecadd_manual =============

Manual memory allocation: success!
Generating '/tmp/nsys-report-1c62.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /home/cuda/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)     Med (ns)    Min (ns)    Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  -----------  ---------  -----------  ------------  ----------------------
     87.0      452,215,583         13  34,785,814.1  2,391,646.0     50,053  351,681,820  96,066,107.9  poll                  
     11.8       61,184,833        546     112,060.1     11,402.0        396   17,274,823     900,088.2  ioctl                 
      0.5        2,606,854          1   2,606,854.0  2,606,854.0  2,606,854    2,606,854   

## **Histogram**

Given an array whose elements are all from a set known a priori, count the occurrencies of each element of the set in the array.

In this example, let the set be the ASCII letters from 'a' to 'z', and the array length be 'n'.

Let's also add the concept of "bins", these are subset of elements that count towards the same total.
For us, letters will be binned in groups of four, that is, 'a-d' will count towards the same total, then 'e-h', and so on.

Key issues to solve:
- data access locality
- output interference

### Wrong (poor memory access efficiency - and data races)

Each thread is given a contigous section of the input.<br>
The input array is evenly partitioned and given to threads, each thread then consumes its partition one element at a time.

Issue:<br>
At a given point in time, all threads will be asking to access values from memory that sit at very distant locations one from the other.
We cannot bundle this data in a single DRAM transaction.

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void histKernel(unsigned char *arr, unsigned int *hist, long n) {
  int part_size = (n + gridDim.x*blockDim.x - 1)/gridDim.x*blockDim.x; // ceil(n / gridDim.x*blockDim.x)
  int tid = (blockIdx.x*blockDim.x + threadIdx.x)*part_size;

  for (int i = tid; i < tid + part_size && i < n; i++) {
    int letter_idx = arr[i] - 97; // 97 is the ASCII code of 'a'
    if (letter_idx >= 0 && letter_idx < 26)
      hist[letter_idx/4] += 1; // divide by 4 to get bins
    }
}

int main() {
  // HP: less threads than elements
  long n = 10000;
  int blockSize = 16;
  int numBlocks = 32;

  unsigned char *arr;
  unsigned int *hist;
  cudaMallocManaged(&arr, n*sizeof(float));
  cudaMallocManaged(&hist, 7*sizeof(float)); // 26 letters in bins of 4 -> 7 bins
  for (int i = 0; i < n; i++)
    arr[i] = (unsigned char)(i%26 + 97);
  for (int i = 0; i < 7; i++)
    hist[i] = 0;

  histKernel<<<numBlocks, blockSize>>>(arr, hist, n);
  cudaDeviceSynchronize();

  cudaFree(arr);
  cudaFree(hist);
  return 0;
}

### Wrong (data races)

Threads process the array in an interleaved fashion.<br>
With the whole grid of threads being smaller than the array, the first thread in the grid processes the first element, the second the second, and so on. Then, the whole grid is "moved" to the right by its own size, therefore the first thread will process element |grid|+0, the second |grid|+1, and so on. They will then move to 2\*|grid|+0, 2\*|grid|+1, ...

Why?<br>
This way, at any point in time, all threads process a **contigous section of elements**. Therefore, **memory accesses are coalesced**, because threads step forward together, and ask for contigous data all at once.
Ultimately, this gives better spatial locality -> multiple piece of data transferred in one DRAM transaction.

There is still one issue: updates to the histogram can happen however they please. This means that two threads could read the same value for 'k = hist[x]' and thus both write 'k + 1', thereby loosing one of the two increments.<br>
This is a classic problem of writing shared memory parallel code: the **data race**.

*Coalesced: DRAM transactions are wide and can move multiple contigous elements at once. If contigous data is accessed by threads in the same **warp**, such data can be moved as part of one same, or very few, transactions.*

*Note: more info on coalescence here: https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/#coalesced-access-to-global-memory*

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void histKernel(unsigned char *arr, unsigned int *hist, long n) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  int stride = blockDim.x*gridDim.x;

  for (int i = tid; i < size; i += stride) {
    int letter_idx = arr[i] - 97;
    if (letter_idx >= 0 && letter_idx < 26)
      hist[letter_idx/4] += 1;
  }
}

// same main as above...

### Atomics

To prevent concurrent updates (read-modify-write operations) to cause data races, we just need to give a thread exclusive access to the modified value from before it reads it until it finishes the update. Essentially, an atomic context.<br>
CUDA gives use ready-to-use primitives for this, at least for most basic operations, therefore, instead of a normal '+1' on the histogram, we need to perform its atomic equivalent 'atomicAdd(...)'.

Issue:<br>
Atomic operations are semantically correct, but take ~1000 cycles to complete on DRAM, , this severely hurts performance.<br>
So, could we get away without atomic operations while preserving correctness? Yes, and as it is often the case, we just need to postpone some part of the program! In this case, we will postpone the global accumulation of histogram bin counts.

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void histKernel(unsigned char *arr, unsigned int *hist, long n) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  int stride = blockDim.x*gridDim.x;

  for (int i = tid; i < size; i += stride) {
    int letter_idx = arr[i] - 97;
    if (letter_idx >= 0 && letter_idx < 26)
      atomicAdd(&(hist[letter_idx/4]), 1);
  }
}

// same main as above...

### Privatization

Create a copy of the histogram counters per block, using shared memory.<br>
Then, rely on atomics on shared memory, since they take ~10 cycles to complete on L2 cache, that is 100x better than DRAM.<br>
As the very last step, accumulate the counters from all copies.

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void histKernel(unsigned char *arr, unsigned int *hist, long n, unsigned int num_bins) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  int stride = blockDim.x*gridDim.x;
  extern __shared__ unsigned int hist_s[];

  for(unsigned int bin_idx = threadIdx.x; bin_idx < num_bins; bin_idx += blockDim.x)
    hist_s[bin_idx] = 0u;

  __syncthreads();

  // compute histogram in shared memory
  for (unsigned int i = tid; i < n; i += stride) {
    int letter_idx = arr[i] - 97;
    if (letter_idx >= 0 && letter_idx < 26)
      atomicAdd(&(hist_s[letter_idx/4]), 1);
  }

  __syncthreads();

  // accumulate between blocks
  for(unsigned int bin_idx = threadIdx.x; bin_idx < num_bins; bin_idx += blockDim.x)
    atomicAdd(&(hist[bin_idx]), hist_s[bin_idx]);
}

// same main as above, just pass '7' as 'num_bins'.

All major problems are now solved! The code could still see some minor optimization, like performing a reduction between the private histogram copies, instead of relying again on atomics, but that is left to the reader.

## **Stencil**

Consider this simple "5-point stencil" kernel on a 2D grid that updates each cell with the average of its four neighbors (up, down, left, right).

Visually:

<img src="https://www.mdpi.com/electronics/electronics-09-01275/article_deploy/html/images/electronics-09-01275-g001-550.jpg" alt="Stencil visualization." width="350" border="2">

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void stencilKernel(float *input, float *out, long m, long n) {
    int x = blockIdx.x*blockDim.x + threadIdx.x;
    int y = blockIdx.y*blockDim.y + threadIdx.y;

    extern __shared__ float tile[];

    int sh_w = blockDim.x + 2;
    // index inside shared memory
    int sh_x = threadIdx.x + 1;
    int sh_y = threadIdx.y + 1;

    // helper function to cleanup index calculations
    auto idx = [&](int yy, int xx) {
      return yy*m + xx;
    };

    // load tile
    if (x < m && y < n)
        tile[sh_y*sh_w + sh_x] = input[idx(y, x)];
    else
        tile[sh_y*sh_w + sh_x] = 0.0f;

    // load halo
    if (threadIdx.x == 0 && x > 0)
        tile[sh_y*sh_w + (sh_x - 1)] = input[idx(y, x - 1)];
    if (threadIdx.x == blockDim.x - 1 && x < m - 1)
        tile[sh_y*sh_w + (sh_x + 1)] = input[idx(y, x + 1)];
    if (threadIdx.y == 0 && y > 0)
        tile[(sh_y - 1) sh_w + sh_x] = input[idx(y - 1, x)];
    if (threadIdx.y == blockDim.y - 1 && y < n - 1)
        tile[(sh_y + 1)*sh_w + sh_x] = input[idx(y + 1, x)];

    // corner halos (not used in 5-point)
    /*
    if (threadIdx.x == 0 && threadIdx.y == 0 && x > 0 && y > 0)
        tile[(sh_y - 1)*sh_w + (sh_x - 1)] = input[idx(y - 1, x - 1)];
    if (threadIdx.x == blockDim.x - 1 && threadIdx.y == 0 && x < m - 1 && y > 0)
        tile[(sh_y - 1)*sh_w + (sh_x + 1)] = input[idx(y - 1, x + 1)];
    if (threadIdx.x == 0 && threadIdx.y == blockDim.y - 1 && x > 0 && y < n - 1)
        tile[(sh_y + 1)*sh_w + (sh_x - 1)] = input[idx(y + 1, x - 1)];
    if (threadIdx.x == blockDim.x - 1 && threadIdx.y == blockDim.y - 1 && x < m - 1 && y < n - 1)
        tile[(sh_y + 1)*sh_w + (sh_x + 1)] = input[idx(y + 1, x + 1)];
    */

    __syncthreads();

    // compute the stencil (5-point average)
    if (x > 0 && x < m - 1 && y > 0 && y < n - 1) {
        float c = tile[sh_y*sh_w + sh_x];
        float l = tile[sh_y*sh_w + (sh_x - 1)];
        float r = tile[sh_y*sh_w + (sh_x + 1)];
        float u = tile[(sh_y - 1)*sh_w + sh_x];
        float d = tile[(sh_y + 1)*sh_w + sh_x];
        out[idx(y, x)] = (c + l + r + u + d)/5.0f;
    } else if (x < m && y < n) {
        out[idx(y, x)] = input[idx(y, x)];
    }
}

int main() {
  long m = 20000;
  long n = 10000;
  dim3 blockSize(32, 32);
  dim3 numBlocks((m + blockSize.x - 1)/blockSize.x, (n + blockSize.y - 1)/blockSize.y);
  size_t shmem = (blockSize.x + 2)*(blockSize.y + 2) * sizeof(float);

  float *input, *out;
  cudaMallocManaged(&input, m*n*sizeof(float));
  cudaMallocManaged(&out, m*n*sizeof(float));
  for (int i = 0; i < m*n; i++)
    input[i] = (float)(i % 100);

  stencilKernel<<<numBlocks, blockSize, shmem>>>(input, out, m, n);
  cudaDeviceSynchronize();

  cudaFree(input);
  cudaFree(out);
  return 0;
}

There is a clear opportunity to exploit shared memory to reuse neighbors inside a block.<br>
Observe the following:
- how many elements end up in the shared memory of each block, and why?
<!--
In general that is (blockSize.x + 2)*(blockSize.y + 2) because each block needs to include one more neighbor along the perimeter, creating an overlap between the shared memory of adjacent blocks (the "halo"). The reason is in the operation itself, each output elements wants four neighbors, regardless of its position.

Note: there are indeed the "corners" in shared memory that don't get used, so we could have 4 less elements in the above calculation. But it's worth "wasting" them to simplify index calculations.
-->
- since the elements in the halo get accessed only by a single thread, could we not store the halo in shared memory and relegate these extra elements that are accessed once to global memory accesses? Would it be a good idea?
<!--
In terms of "can", it is absolutely possible.
It is a bad idea, loosing a lot of performance because:
- now threads that are on the edge of each block will diverge from the others in their warp; moreover it diverges over a very costly global memory access.
- global memory accesses are slow and expensive, if we perform them in bulk during the preparation of shared memory we can save time as they get coalesced!

Remember: shared memory isn't only for reuse, but also to improve memory access performance and exploit coalesced memory acceses on DRAM.

Another observation: the "wasted" shared memory equals the perimeter of the input, this when the input size grows, it grows quadratically, while the wasted memory grows linearly with the perimeter, therefore at scale it is a negligible overhead.

More info on coalescence here: https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/#coalesced-access-to-global-memory
-->
- what would happen if we remove the `__synchthreads()` instruction?
<!--
Race conditions! We could have a thread in the block start reading addresses in shared memory that others had to write, before they have been written.

In other words: we could violate a read-after-write data dependency.
-->
- what would we need to do if we wanted to have the input grid be updated in-place instead of relying on another output buffer?
<!--
Let each thread store its own result in a register, synch threads globally (incrementing with a CAS a global variable and spinning on it until it reaches the thread count) and only then let each thread overwrite the input.
Improvement:
The global variable could count blocks, not threads, and the first thread of each block can increment it as soon as its block has synched after preparing its shared memory. That is because after shared memory has been loaded in all blocks, subsequent reads will only hit shared memory, and there will be no more race conditions.
-->

*Note: the term to indicate the part of the input that overlaps between neighboring blocks is "**halo**". Yes Chief, like the videogame.*

## **MatMul**

This is the classic idea of a matrix multiplication:

<!--<img src="https://upload.wikimedia.org/wikipedia/commons/a/ad/Matrix_Multiplication.png" alt="Matrix multiplication overview." width="350" border="0">-->
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/Matrix_multiplication_diagram.svg/1024px-Matrix_multiplication_diagram.svg.png" alt="Matrix multiplication overview." width="350" border="0">

Key goal: parallelize the workload and exploit data reuse through **tiling**.

The term tile draws on the analogy that a large wall (e.g., the global memory data) can be covered by small tiles (e.g., subsets that can each fit into the shared memory).
An important criterion is that the kernel computation on these tiles can be done independently of each other.
Note that, for this reason, not all data structures can be partitioned into tiles, given an arbitrary kernel function.

In particular we will rely on:
- tiling to use [for reuse] shared memory in the output;
- strip mining to use [for reuse] shared memory while sliding over input operands by breaking the inner dimension's loop in tiles and go over them one by one;

<br>For these examples, let us assume three square matrices of identical size: "width" in the code.

### No Tiling

No optimization of sorts, this is just a basic matmul kernel very much like what you would write to run on CPU.

In [ ]:
%%writefile only_for_highlight.cpp

__global__
void matMul(float* M, float* N, float* P, int width) {
  int row = blockIdx.y*blockDim.y + threadIdx.y; // row of M and P
  int col = blockIdx.x*blockDim.x + threadIdx.x; // col of N and P

  if ((row < width) && (col < width)) {
    float acc = 0;
    for (int k = 0; k < width; ++k) // k is the col of M and row of N
      acc += M[row*width + k]*N[k*width + col];
    P[row*width + col] = acc;
  }
}

// launched with:
int main() {
  int width = ...;
  int block_size = ...;
  float M[width*width], N[width*width], P[width*width];
  //...
  matMul<<<dim3(block_size, block_size), dim3(width/block_size, width/block_size)>>>(M, N, P, width);
  //...
}

<a name="tiled-mm"></a>
### Tiled Matrix Multiplication


Each block is assigned an output tile. Then, to move its threads along in the span of the other two operands in clear steps, progressively reducing the output elements in the assigned tile!<br>
This is a technique called **strip-mining**, which takes a long-running loop and break it into phases.
Each phase involves an inner loop that executes a few consecutive iterations of the original loop.
The original loop becomes an outer loop whose role is to iteratively invoke the inner loop so that all the iterations of the original loop are executed in their original order.
By adding barrier synchronizations before and after the inner loop, we force all threads in the same block to focus their work on the same section of input data during each phase.
Strip-mining is an important means to creating the phases that are needed by tiling in data parallel programs.

<img src="https://i.imgur.com/VGNlxxx.png" alt="CUDA memory hierarchy overview." width="400" border="2">

*Credit: "Programming Massiveli Parallel Processors" by W. Hwu, D. Kirk, I. Hajj*

In other words, we have a one-thread-one-output-element relation, then each thread iterates over additional tiles formed along the width and height of the other two operands. For each tile, the block loads it in its shared memory and each threads reuses it. Continuing until after all tiles each output element has been fully reduced.

<!--<img src="https://polimi365-my.sharepoint.com/:i:/r/personal/10669641_polimi_it/Documents/Images/matmul_reuse.png" alt="Reuse in shared memory for a tiled matrix multiplication." width="500" border="2">-->
<img src="https://i.imgur.com/TdkJXHa.png" alt="Reuse in shared memory for a tiled matrix multiplication." width="400" border="2">

You should verify that the potential reduction in global memory traffic in a square matrix multiplication is proportional to the dimension of the blocks that are used.
With $width{\times}width$ blocks, the potential reduction of global memory traffic should be $width$.
That is, if we use 16$\times$16 blocks, we can potentially reduce the global memory traffic to 1/16 of the original level through collaboration between threads.
<!--
Another way to look at it: with tiles of 16x16, each value of those tiles is loaded once, and loaded again only 1/16th of the times it was before, because before needing to realod it the whole tile needs to have moved and come back.
Alternatively, see it as the whole 16 accesses to a value that happen inside a tile now costing only 1 global memory access.
-->

A fairly universal approach to calculate the reduction in required global memory bandwidth is (# accesses to elements in the tile before its overwritten)/(# of elements in the tile). Where the denominator is the global accesses required to load the tile and the numerator the accesses that we would have needed if it were not for shared memory.

*Note: tile size is by no means constrainted to be equal to block size. However, for sake of providing a simple example, the following code assumes that tile size = block size.*

In [ ]:
%%writefile only_for_highlight.cpp

#define TILE_DIM 16

__global__
void matMul(float* M, float* N, float* P, int width) {
  __shared__ float M_s[TILE_DIM][TILE_DIM];
  __shared__ float N_s[TILE_DIM][TILE_DIM];

  int row = blockIdx.y*TILE_DIM + threadIdx.y;
  int col = blockIdx.x*TILE_DIM + threadIdx.x;

  float acc = 0;
  for (int t = 0; t < (width + TILE_DIM - 1)/TILE_DIM; t++) { // allow for 'width's not multiples of 'TILE_DIM' and viceversa
    // shared memory loads
    if (t*TILE_DIM + threadIdx.x < width && row < width)
      M_s[threadIdx.y][threadIdx.x] = M[row*width + t*TILE_DIM + threadIdx.x];
    else
      M_s[threadIdx.y][threadIdx.x] = 0.0f;

    if (t*TILE_DIM + threadIdx.y < width && col < width)
      N_s[threadIdx.y][threadIdx.x] = N[(t*TILE_DIM + threadIdx.y)*width + col];
    else
      N_s[threadIdx.y][threadIdx.x] = 0.0f;

    __syncthreads();

    // compute
    for (int k = 0; k < TILE_DIM; k++)
      acc += M_s[threadIdx.y][k] * N_s[k][threadIdx.x];

    __syncthreads();
  }

  // write result
  if (row < width && col < width)
    C[row*width + col] = acc;
}

// launched with:
int main() {
  int width = ...;
  float M[width*width], N[width*width], P[width*width];
  //...
  matMul<<<dim3(width/TILE_DIM, width/TILE_DIM), dim3(TILE_DIM TILE_DIM)>>>(M, N, P, width);
  //...
}

### Depth Parallel (tiling the inner dimension and reducing)

Now, let us look at the matrix multiplication as a 3D tensor, width and height are those of the output, while depth is the inner dimension, each "cell" in the tensor is a multiply-and-accumulate (pruduct + sum) to be done, while depth-wise 1x1 tensors pass through the values to be accumulated for one output.

In practive, we expand spatial parallelism (one thread per output element) with depth parallelism (multiple threads collaborating along the inner accumulation dimension, i.e. 'k').

Each output element is now computed by multiple threads, each handling a tile of the inner dimension (those that were strip-mined sequentially before), and the partial sums are then reduced efficiently within the shared memory of the cooperative thread group.

In [ ]:
%%writefile only_for_highlight.cpp
#include <stdio.h>

#define TILE_DIM 16 // tile size for output width and height dimensions
#define DEPTH_THREADS 8 // number of tiles along the inner dimension

__global__
void matMul(float* M, float* N, float* C, int width) {
    __shared__ float sh[TILE_DIM][TILE_DIM][DEPTH_THREADS];

    int row = blockIdx.y*TILE_DIM + threadIdx.y;
    int col = blockIdx.x*TILE_DIM + threadIdx.x;
    int depth_id = threadIdx.z; // which slice of the inner dimension this thread handles

    float sum = 0.0f;

    // each depth thread handles a subset of k values: strided by DEPTH_THREADS
    for (int k = depth_id; k < width; k += DEPTH_THREADS) {
        float a = (row < width && k < width) ? M[row*width + k] : 0.0f;
        float b = (col < width && k < width) ? N[k*width + col] : 0.0f;
        sum += a * b;
    }

    // store partial sum
    if (row < width && col < width)
        sh[threadIdx.y][threadIdx.x][depth_id] = sum;

    __syncthreads();

    // reduction across DEPTH_THREADS
    if (depth_id == 0 && row < width && col < width) {
        float acc = 0.0f;
        for (int d = 0; d < DEPTH_THREADS; ++d)
            acc += partial[threadIdx.y][threadIdx.x][d];
        C[row * width + col] = acc;
    }
}

// launched with:
int main() {
  int width = ...;
  float M[width*width], N[width*width], P[width*width];
  //...
  dim3 block(TILE_DIM, TILE_DIM, DEPTH_THREADS);
  dim3 grid((width + TILE_DIM - 1)/TILE_DIM, (width + TILE_DIM - 1)/TILE_DIM); // 2D grid of 3D blocks
  matMul<<<grid, block>>>(M, N, P, width);
  //...
}


Ultimately, this by itself is **far worse** than tiling!<br>
Why: much less data reuse, each thread fetches from M and N independently, tiles are not shared.

Modern libraries like cuBLAS combine a slight bit of this depth parallelism with the previous tiling method, mainly to hide the memory latency of loading data in shared memory by doing it alternatingly across a few different depth tiles (some load data while others compute).

## **Extras**

### Occupancy Calculation

Several API functions exist to assist programmers in choosing thread block size and cluster size based on register and shared memory requirements.

Here we showcase the "occupancy calculator" API, `cudaOccupancyMaxActiveBlocksPerMultiprocessor`, can provide an occupancy prediction based on the block size and shared memory usage of a kernel. This function reports occupancy in terms of the number of concurrent thread blocks per multiprocessor.

In [ ]:
%%writefile occupancy_calculator.cpp
__global__
void MyKernel(int *d, int *a, int *b) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  d[idx] = a[idx] * b[idx];
}

int main() {
  int numBlocks; // occupancy in terms of active blocks
  int blockSize = 32;
  // these variables are used to convert occupancy to warps
  int device;
  cudaDeviceProp prop;
  int activeWarps;
  int maxWarps;

  cudaGetDevice(&device);
  cudaGetDeviceProperties(&prop, device);
  cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, MyKernel, blockSize, 0);

  activeWarps = numBlocks * blockSize / prop.warpSize;
  maxWarps = prop.maxThreadsPerMultiProcessor / prop.warpSize;
  std::cout << "Occupancy: " << (double)activeWarps / maxWarps * 100 << "%" << std::endl;

 return 0;
}

### Debugging CUDA with GDB

Well, everyone needs to debug sometimes! Here's how to do it when CUDA is in the picture as well:

In [ ]:
!apt-get install -y cuda-toolkit-12-5

In [ ]:
%%writefile red_flag.cpp
#include<stdio.h>

__global__
void susKernel(const float* __restrict__ in, float* __restrict__ out, int n) {
  extern __shared__ float s[];
  int t_id = threadIdx.x;
  int g_t_id = blockIdx.x*blockDim.x + t_id;

  s[t_id] = in[g_t_id];
  __syncthreads();

  for (int stride = blockDim.x; stride > 0; stride /= 2) { // bug: stride should start at blockDim.x/2...
    if (t_id < stride)
      s[t_id] += s[t_id + stride];
    __syncthreads();
  }
}

int main() {
  const int n = 4096;
  float h_in[n], h_out[n];
  for (int i = 0; i < n; i++)
    h_in[i] = i + 1;

  float *d_in, *d_out;
  cudaMalloc(&d_in, n*sizeof(float));
  cudaMalloc(&d_out, n*sizeof(float));
  cudaMemcpy(d_in, h_in, n*sizeof(float), cudaMemcpyHostToDevice);

  int blockSize = 256;
  int gridSize = (n + blockSize - 1)/blockSize;
  size_t shmem = blockSize * sizeof(float);
  susKernel<<<gridSize, blockSize, shmem>>>(d_in, d_out, n);
  cudaMemcpy(h_out, d_out, n*sizeof(float), cudaMemcpyDeviceToHost);

  cudaFree(d_in);
  cudaFree(d_out);
  return 0;
}


Writing red_flag.cpp


In [ ]:
!mv red_flag.cpp red_flag.cu
!nvcc -G -g red_flag.cu -o red_flag

In [ ]:
%%bash
# Colab does give easy access to an interactive console, so we need to "pre-program" our debug session...
# This simple example just runs the debugger
cat > gdbcmds.txt <<'EOF'
run
quit
EOF

set +e
cuda-gdb --batch --command=gdbcmds.txt ./red_flag
echo "cuda-gdb exited with status $?"

[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
[New Thread 0x1555535ff000 (LWP 7486)]
[New Thread 0x1555527ff000 (LWP 7487)]
[Detaching after fork from child process 7488]
[New Thread 0x155551383000 (LWP 7497)]
[Thread 0x1555527ff000 (LWP 7487) exited]
[Thread 0x1555535ff000 (LWP 7486) exited]
[Thread 0x1555552cf000 (LWP 7482) exited]
[Thread 0x155551383000 (LWP 7497) exited]
[New process 7482]
[Inferior 1 (process 7482) exited normally]
cuda-gdb exited with status 0


Aaaaand, this is as far as we can go on Colab.<br>
To debug inside the kernel we would need a local machine to run the 'cuda-gdbserver' with access to the GPU.<br>
If you are curious, the docs are here: https://docs.nvidia.com/cuda/cuda-gdb/

### Yet Another Simple Exercise on Divergence

A [tiled matrix multiplication](#tiled-mm) kernel has many boundary condition checks, such as the ones to load M tiles in the following code snippet:

In [ ]:
%%writefile only_for_highlight.cpp

#define TILE_WIDTH 16

//...
for (int t = 0; t < (width + TILE_DIM - 1)/TILE_DIM; t++) {
  if (row < width && t*TILE_DIM + threadIdx.x < width) {
    M_s[threadIdx.y][threadIdx.x] = M[row*width + p*TILE_DIM + threadIdx.x];
  } else {
    M_s[threadIdx.y][threadIdx.x] = 0.0;
  }
  //...
}

Assuming 100x100 square matrices, 16x16 square tiles, and 16x16 thread blocks, estimate what will be the overall impact of control divergence in this case.
<!--
Ah-aH! You wanted a solution huh? Sorry, it's 3am and I am too tired for this...
Please try to build some confidence in yourself and trust your own solution.
If self-confidence is not an option, try harder.
If you already tried your hardest, send me and email and I will come up with the solution then ^-^!
-->

### Tiled Arbitrary Matmul

Example of how to handle general rectangular matrices with shared memory.<br>
Elements outside the not-fully overlapping tiles become 0.

Source: https://stackoverflow.com/a/18856054/12501261

*Note: on small matrices, between with or wihout shared memory we will not likely observe any improvement. That is because, for the GPU architecture we are running, the L1 cache already "does the job" of the shared memory. Remember that, apart from very old architectures, shared memory can be seen as a controlled cache.*

In [ ]:
%%writefile arbitrary_matmul.cpp
#include<stdio.h>

#define SCALE 1000000.0
#define TILE_DIM 10

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

// in reality ARows = CRows, ACols = BRows, BCols = CCols, but let's keep the separate for sake of clarity
__global__
void matMul(float* A, float* B, float* C, int ARows, int ACols, int BRows, int BCols, int CRows, int CCols) {
  float CValue = 0;

  int Row = blockIdx.y*TILE_DIM + threadIdx.y; // aka 'm'
  int Col = blockIdx.x*TILE_DIM + threadIdx.x; // aka 'n'

  __shared__ float As[TILE_DIM][TILE_DIM];
  __shared__ float Bs[TILE_DIM][TILE_DIM];

  // coalesce memory operations through tiled loads in shared memory
  for (int k = 0; k < (TILE_DIM + ACols - 1)/TILE_DIM; k++) {
    if (k*TILE_DIM + threadIdx.x < ACols && Row < ARows)
      As[threadIdx.y][threadIdx.x] = A[Row*ACols + k*TILE_DIM + threadIdx.x];
    else
      As[threadIdx.y][threadIdx.x] = 0.0;

    if (k*TILE_DIM + threadIdx.y < BRows && Col < BCols)
      Bs[threadIdx.y][threadIdx.x] = B[(k*TILE_DIM + threadIdx.y)*BCols + Col];
    else
      Bs[threadIdx.y][threadIdx.x] = 0.0;

    __syncthreads();

    // compute
    for (int k = 0; k < TILE_DIM; ++k)
      CValue += As[threadIdx.y][k] * Bs[k][threadIdx.x];

    __syncthreads();
  }

  // write result
  if (Row < CRows && Col < CCols)
    C[Row*CCols + Col] = CValue;
}

void initMatrixRand(float *M, int r, int c) {
  for (int i = 0; i < r; i++)
    for (int j = 0; j < c; j++)
      M[i*c + j] = (float)rand() / SCALE;
}

void initMatrixValue(float v, float *M, int r, int c) {
  for (int i = 0; i < r; i++)
    for (int j = 0; j < c; j++)
      M[i*c + j] = v;
}

bool checkMatMul(float *A, float *B, float *C, int m, int k, int n) {
  for (int i = 0; i < m; i++) {
    for (int j = 0; j < n; j++) {
      float acc = 0.0;
      for (int l = 0; l < k; l++)
        acc += A[i*k + l] * B[l*n + j];
      // floats are not associative, we need to allow some margin!
      if (C[i*n + j] > acc*1.001f || C[i*n + j] < acc*0.999f) {
        printf("Error in position (%d, %d), expected %f, got %f!\n", i, j, acc, C[i*n + j]);
        return false;
      }
    }
  }
  return true;
}

int main() {
  int m = 1000; // ARows = CRows
  int k = 500;  // ACols = BRows (inner dimension)
  int n = 700;  // BCols = CCols
  dim3 blockSize(TILE_DIM, TILE_DIM);
  dim3 numBlocks((n + TILE_DIM - 1)/TILE_DIM, (m + TILE_DIM - 1)/TILE_DIM);

  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  float *A;
  float *B;
  float *C;

  // possible improvement: store "B/Bs" in column-major order, while "A/As"
  // stays row-major, because the innermost loop is on "k"!
  cudaMallocManaged(&A, m*k*sizeof(float));
  cudaMallocManaged(&B, k*n*sizeof(float));
  cudaMallocManaged(&C, m*n*sizeof(float));

  initMatrixRand(A, m, k);
  initMatrixRand(B, k, n);
  initMatrixValue(0.0f, C, m, n);

  int device;
  cudaGetDevice(&device);

  cudaMemPrefetchAsync(A, m*k*sizeof(float), device);
  cudaMemPrefetchAsync(B, k*n*sizeof(float), device);
  //cudaMemPrefetchAsync(C, m*n*sizeof(float), device);

  cudaEventRecord(start);
  matMul<<<numBlocks, blockSize>>>(A, B, C, m, k, k, n, m, n);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  gpuErrchk(cudaDeviceSynchronize());

  cudaMemPrefetchAsync(C, m*n*sizeof(float), cudaCpuDeviceId);
  gpuErrchk(cudaDeviceSynchronize());

  if (checkMatMul(A, B, C, m, k, n))
    printf("Result OK!\n");
  else
    printf("Result NOT OK!\n");

  printf("Execution time (ms): %f \n", milli);

  cudaEventDestroy(stop);
  cudaFree(A);
  cudaFree(B);
  cudaFree(C);

  return 0;
}

Writing arbitrary_matmul.cpp


In [ ]:
!mv arbitrary_matmul.cpp arbitrary_matmul.cu
!nvcc -arch=sm_75 arbitrary_matmul.cu -run

Result OK!
Execution time (ms): 4.256256 


### An Improbable Divergence Scenario

Given the following kernel, estimate what impact will have the serialization due to control divergence.

In [ ]:
%%writefile only_for_highlight.cpp

#define N 1024

__global__ void loopKernel(float* v, float* o, int n) {
  int g_id = blockIdx.x*blockDim.x + threadIdx.x;
  int delta = g_id % 2;

  o[g_id] = 0;
  for (int i = -delta; i <= delta; i += 1) {
    o[g_id] += 2*v[g_id + i];
  }
}

// launched with:
int main() {
  float v[N], o[N];
  //...
  loopKernel<<<4, N/4>>>(v, o, N);
  //...
}

This code is very unlikely, but for the sake of an example, think of it as a weird take on a 1D convolution.

How to improve it up is obvious: write two kernels, one for even and one for odd position elements!

In [ ]:
%%writefile only_for_highlight.cpp

#define N 1024

__global__ void loopKernelEven(float* v, float* o, int n) {
  int g_id = (blockIdx.x*blockDim.x + threadIdx.x)*2;

  o[g_id] = 0;
  for (int i = -1; i <= 1; i += 1) {
    o[g_id] += 2*v[g_id + i];
  }
}

__global__ void loopKernelOdd(float* v, float* o, int n) {
  int g_id = (blockIdx.x*blockDim.x + threadIdx.x)*2 + 1;

  o[g_id] += 2*v[g_id];
}

// launched with:
int main() {
  float v[N], o[N];
  //...
  loopKernelEven<<<4, N/8>>>(v, o, N);
  loopKernelOdd<<<4, N/8>>>(v, o, N);
  //...
}

### Warp Shuffles for Reduction

Let us improve the reduction!

- classic version: each block moves a part of the input in shared memory, reduces it one logarithmic tree level at a time (with syncs in between), and then does the final reduction across blocks;
- shuffle version: each block moves a part of the input in shared memory, each warp reduces internally its set of values, then we go at the block level, and finally across blocks;

In [ ]:
%%writefile reduce.cpp
#include <stdio.h>
#include <stdlib.h>

#define WARP_SIZE 32

// classic shared-memory reduction kernel
__global__
void blockReduceClassic(const float* __restrict__ g_in, float* __restrict__ g_out, size_t n) {
  extern __shared__ float sdata[];
  unsigned int tid = threadIdx.x;
  unsigned int idx = blockIdx.x * blockDim.x * 2 + tid;

  float val = 0.0f;
  if (idx < n)
    val = g_in[idx];
  if (idx + blockDim.x < n)
    val += g_in[idx + blockDim.x];

  sdata[tid] = val;
  __syncthreads();

  for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
    if (tid < s) // active threads: first 's' of each block
      sdata[tid] += sdata[tid + s];
    __syncthreads();
  }

  if (tid == 0)
    g_out[blockIdx.x] = sdata[0];
}

// warp shuffle helper
__inline__ __device__
float warpReduceSum(float val) {
  unsigned mask = 0xffffffffu; // all 1s, all active
  for (int offset = warpSize / 2; offset > 0; offset >>= 1)
    val += __shfl_down_sync(mask, val, offset);
  return val;
}

// shuffle-based reduction kernel
__global__
void blockReduceShuffle(const float* __restrict__ g_in, float* __restrict__ g_out, size_t n) {
  extern __shared__ float sdata[];
  unsigned int tid = threadIdx.x;
  unsigned int lane = tid % warpSize; // position (lane) inside warp
  unsigned int warp_id = tid / warpSize;

  size_t idx = (size_t)blockIdx.x * blockDim.x * 2 + tid;

  float sum = 0.0f;
  if (idx < n)
    sum = g_in[idx];
  if (idx + blockDim.x < n)
    sum += g_in[idx + blockDim.x];

  sum = warpReduceSum(sum);

  if (lane == 0)
    sdata[warp_id] = sum;
  __syncthreads();

  if (warp_id == 0) {
    float val = (lane < (blockDim.x / warpSize)) ? sdata[lane] : 0.0f;
    val = warpReduceSum(val);
    if (lane == 0)
      g_out[blockIdx.x] = val;
  }
}

// shuffle version (host entry point)
float reduceShuffle(const float* d_in, size_t n) {
  int threads = 1024;
  int blocks = (int)((n + threads * 2 - 1) / (threads * 2)); // 2 elements per thread => each thread does 1 sum in the first level
  float* d_out;
  cudaMalloc(&d_out, blocks * sizeof(float));

  size_t shmem = (threads / WARP_SIZE) * sizeof(float);
  blockReduceShuffle<<<blocks, threads, shmem>>>(d_in, d_out, n);

  float* h_out = (float*)malloc(blocks * sizeof(float));
  cudaMemcpy(h_out, d_out, blocks * sizeof(float), cudaMemcpyDeviceToHost);

  float sum = 0.0f;
  for (int i = 0; i < blocks; i++)
    sum += h_out[i];

  free(h_out);
  cudaFree(d_out);
  return sum;
}

// classic version (host entry point)
float reduceClassic(const float* d_in, size_t n) {
  int threads = 1024;
  int blocks = (int)((n + threads * 2 - 1) / (threads * 2));
  float* d_out;
  cudaMalloc(&d_out, blocks * sizeof(float));

  size_t shmem = threads * sizeof(float);
  blockReduceClassic<<<blocks, threads, shmem>>>(d_in, d_out, n);

  float* h_out = (float*)malloc(blocks * sizeof(float));
  cudaMemcpy(h_out, d_out, blocks * sizeof(float), cudaMemcpyDeviceToHost);

  float sum = 0.0f;
  for (int i = 0; i < blocks; i++)
    sum += h_out[i];

  free(h_out);
  cudaFree(d_out);
  return sum;
}

int main() {
  size_t N = 1 << 30;
  float* h_in = (float*)malloc(N * sizeof(float));
  for (size_t i = 0; i < N; i++)
    h_in[i] = 1.0f;

  float* d_in;
  cudaMalloc(&d_in, N * sizeof(float));
  cudaMemcpy(d_in, h_in, N * sizeof(float), cudaMemcpyHostToDevice);

  // timing
  float milli;
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);
  float sumClassic = reduceClassic(d_in, N);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Execution classic time (ms): %f \n", milli);

  cudaEventRecord(start);
  float sumShuffle = reduceShuffle(d_in, N);
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&milli, start, stop);
  printf("Execution shuffle time (ms): %f \n", milli);

  printf("Computed values:\nExpected: %.0f\n", (float)N);
  printf("Classic:  %.0f\n", sumClassic);
  printf("Shuffle:  %.0f\n", sumShuffle);

  cudaEventDestroy(start);
  cudaEventDestroy(stop);
  cudaFree(d_in);
  free(h_in);
  return 0;
}


Writing reduce.cpp


In [ ]:
!mv reduce.cpp reduce.cu
!nvcc -arch=sm_75 reduce.cu -run

Execution classic time (ms): 85.974594 
Execution shuffle time (ms): 35.604031 
Computed values:
Expected: 1073741824
Classic:  1073741824
Shuffle:  1073741824


### Jacobi Iteration Method for the Laplace Equation

The Laplace equation is a partial differential equation (PDE) in the form:

$\triangledown^2 u(x, y) = 0\;\; \Leftrightarrow \;\;\frac{\delta^2d}{\delta x^2} + \frac{\delta^2d}{\delta y^2} = 0$

And arises in many physical contexts, like steady-state heat conduction and fluid flow.

We want to solve it numerically, and for that, we discretize $x$ and $y$ in a uniform grid with spacing $h$, and we approximate the laplacian at each interior point via finite differences w.r.t. its neighbors:

$\triangledown^2 u(x_i, y_j) \approx \frac{u_{i-1, j} + u_{i+1, j} + u_{i, j-1} + u_{i, j+1} - 4u_{i,j}}{h^2}$

Imposing $\triangledown^2 u(x, y) = 0$ thus gives us a simple dependency of each point on its four neighbors:

$u_{i, j} = \frac{1}{4} (u_{i-1, j} + u_{i+1, j} + u_{i, j-1} + u_{i, j+1})$

In other words, we have a solution when the value at each interior point equals the average of its four neighbors.

The **Jacobi method** is a simple iterative technique to approximate solutions of linear systems, ad we will use it to find the steady-state above.<br>
Applied to the Laplace equation:
- start with an initial guess $u^{(0)}$
- at each iteration, compute a new solution $u^{(k + 1)}$ by replacing every interior grid point with the average of its neighbors from the previous iteration
- keep the boundary values fixed (these are the Dirichlet boundary conditions)
- continue until convergence to a steady-state



In [ ]:
%%writefile jacobi.cpp
#include <cstdio>
#include <cstdlib>
#include <cmath>

__global__
void lap(int I, int J, const float* __restrict__ u1, float* __restrict__ u2) {
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  int j = threadIdx.y + blockIdx.y * blockDim.y;

  if (i >= I || j >= J)
    return;
  int id = i + j * I;

  if (i == 0 || i == I - 1 || j == 0 || j == J - 1) {
  // boundary: Dirichlet condition, keep fixed
    u2[id] = u1[id];
  } else {
    // interior: average of neighbors
    u2[id] = 0.25f * (u1[id - 1] + u1[id + 1] +
    u1[id - I] + u1[id + I]);
  }
}

int main() {
    const int I = 128;        // grid width
    const int J = 128;        // grid height
    const int N = I * J;      // total points
    const int max_iter = 500; // fixed iteration count

    float *h_u = (float*)malloc(N * sizeof(float));

    // initialization: boundary = 0, interior = random values
    for (int j = 0; j < J; j++) {
        for (int i = 0; i < I; i++) {
            int id = i + j * I;
            if (i == 0 || i == I-1 || j == 0 || j == J-1)
                h_u[id] = 0.0f;
            else
                h_u[id] = static_cast<float>(rand()) / RAND_MAX;
        }
    }

    float *d_u1, *d_u2;
    cudaMalloc(&d_u1, N * sizeof(float));
    cudaMalloc(&d_u2, N * sizeof(float));

    cudaMemcpy(d_u1, h_u, N * sizeof(float), cudaMemcpyHostToDevice);

    dim3 blockDim(16, 16);
    dim3 gridDim((I + blockDim.x - 1)/blockDim.x, (J + blockDim.y - 1)/blockDim.y);

    // Jacobi iterations
    for (int iter = 0; iter < max_iter; iter++) {
        lap<<<gridDim, blockDim>>>(I, J, d_u1, d_u2);
        cudaDeviceSynchronize();

        // NOTE: we can't update locations in place, since other threads will be using them at the same time!
        // => swap pointers (ping-pong)
        float* tmp = d_u1;
        d_u1 = d_u2;
        d_u2 = tmp;
    }

    cudaMemcpy(h_u, d_u1, N * sizeof(float), cudaMemcpyDeviceToHost);

    printf("Result snapshot:\n");
    for (int j = 0; j < 5; j++) {
        for (int i = 0; i < 5; i++) {
            printf("%6.3f ", h_u[i + j * I]);
        }
        printf("\n");
    }

    free(h_u);
    cudaFree(d_u1);
    cudaFree(d_u2);

    return 0;
}


In [ ]:
!mv jacobi.cpp jacobi.cu
!nvcc -arch=sm_75 jacobi.cu -run

Extra:

Check for convergence, every $K$ Jacobi iterations run a second kernel that checks for convergence between 'd_u1' and 'd_u2'.
The check simply consist of finding, across all locations, the one with the maximum diference between 'd_u1' and 'd_u2', if that difference is less than a user-define tolerance, then terminate.
In other words, this is just a reduce for the maximum value.

In [ ]:
%%writefile highlight_only.cpp

#define K 10

__global__
void max_diff(int N, const float* u1, const float* u2, float* d_out) {
  __shared__ float sdata[256];
  int tid = threadIdx.x;
  int idx = blockIdx.x * blockDim.x + tid;

  float diff = 0.0f;
  if (idx < N)
    diff = fabsf(u1[idx] - u2[idx]);
  sdata[tid] = diff;
  __syncthreads();

  // this is the classic shared memory reduce
  for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
    if (tid < stride) {
      if (sdata[tid + stride] > sdata[tid])
        sdata[tid] = sdata[tid + stride];
    }
      __syncthreads();
  }

  if (tid == 0) d_out[blockIdx.x] = sdata[0];
}

int main() {
  // ... skipping code from before ...

  float tolerance = 1e-4f;
  float *d_blockmax, *h_blockmax;
  int blocks = (N + 255) / 256;
  h_blockmax = (float*)malloc(blocks * sizeof(float));
  cudaMalloc(&d_blockmax, blocks * sizeof(float));

  for (int iter = 0; iter < max_iter; iter++) {
    lap<<<gridDim, blockDim>>>(I, J, d_u1, d_u2);
    cudaDeviceSynchronize();

    // every K iterations, convergence check
    if (iter % K == 0) {
      max_diff<<<blocks, 256>>>(N, d_u1, d_u2, d_blockmax);
      cudaMemcpy(h_blockmax, d_blockmax, blocks*sizeof(float), cudaMemcpyDeviceToHost);

      // finish the reduce on the host
      float maxdiff = 0.0f;
      for (int b = 0; b < blocks; b++)
        if (h_blockmax[b] > maxdiff)
          maxdiff = h_blockmax[b];

      if (maxdiff < tolerance) {
        printf("Converged at iteration %d with diff=%e\n", iter, maxdiff);
        break;
      }
    }

    // swap pointers
    float* tmp = d_u1;
    d_u1 = d_u2;
    d_u2 = tmp;
  }

  // ... skipping code from before ...
}

## **Colab-to-PDF**

Welcome to a graveyard of pain and suffering...

In [ ]:
%%capture
!pip install -q "nbconvert[webpdf]" playwright
!playwright install chromium

In [ ]:
# mount Google Drive
from google.colab import drive, files
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
# simple version

# ISSUE: font is too large
#!jupyter nbconvert --to webpdf "CUDA.ipynb" --allow-chromium-download --embed-images
#files.download("CUDA.pdf")

# ISSUE: itemizes without an empty before them fail to get converted, same goes for HTML linebreaks "<br>"
#!jupyter nbconvert --to latex "CUDA.ipynb" --embed-images
#files.download("CUDA.tex")

# ISSUE: wants reveal.js
#!jupyter nbconvert --to slides "CUDA.ipynb" --embed-images
#files.download("CUDA.slides.html")

In [ ]:
# webpdf version with images included
# ISSUE: font is too large

import nbformat, os, re, requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

notebook_path = "CUDA.ipynb"

with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

os.makedirs("images", exist_ok=True)

def download_image(url, folder="images"):
    """Download any image (even without file extension)."""
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        name = os.path.basename(urlparse(url).path) or "image"
        if not re.search(r'\.\w{2,4}$', name):
            ext = r.headers.get("content-type", "").split("/")[-1]
            if ext in ["jpeg", "jpg", "png", "gif", "svg", "webp"]:
                name += f".{ext}"
            else:
                name += ".png"
        local_path = os.path.join(folder, name)
        with open(local_path, "wb") as f:
            f.write(r.content)
        return local_path
    except Exception as e:
        print(f"Could not fetch {url}: {e}")
        return None

for cell in nb.cells:
    if cell.cell_type in ("markdown", "raw"):
        soup = BeautifulSoup(cell.source, "html.parser")
        modified = False
        for img in soup.find_all("img", src=True):
            src = img["src"]
            if src.startswith("http"):
                local = download_image(src)
                if local:
                    img["src"] = local
                    modified = True
        if modified:
            cell.source = str(soup)

local_nb = "CUDA_embedded.ipynb"
with open(local_nb, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

!jupyter nbconvert --to webpdf "CUDA_embedded.ipynb" --allow-chromium-download

files.download("CUDA_embedded.pdf")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
Could not fetch https://polimi365-my.sharepoint.com/:i:/r/personal/10669641_polimi_it/Documents/Images/CUDA_memory_model.PNG?csf=1&web=1&e=JIHi6R: 403 Client Error: FORBIDDEN for url: https://polimi365-my.sharepoint.com/personal/10669641_polimi_it/Documents/Images/CUDA_memory_model.PNG?csf=1&web=1&e=JIHi6R&CID=b54f4138-efcf-4a0b-9852-c03228147079
Could not fetch https://polimi365-my.sharepoint.com/:i:/r/personal/10669641_polimi_it/Documents/Images/blocks_over_input.png: 403 Client Error: FORBIDDEN for url: https://polimi365-my.sharepoint.com/personal/10669641_polimi_it/Documents/Images/blocks_over_input.png?CID=0e146fc5-4a2d-4873-b0ac-e3efeb9bbbd0
[NbConvertApp] Converting notebook CUDA_embedded.ipynb to webpdf
[NbConvertApp] Building PDF
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 418575 bytes to CUDA

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# latex (ALMOST) completely working version

import nbformat, re, os, time, requests
from bs4 import BeautifulSoup

notebook_path = "CUDA.ipynb"
tmp_path = "/content/CUDA_tmp"
images_dir = "/content/images"

os.makedirs(images_dir, exist_ok=True)

with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

for cell in nb.cells:
    if cell.cell_type == "markdown":
        text = cell.source

        # replace HTML <br> tags with real newlines
        text = re.sub(r'<br\s*/?>', '\n', text, flags=re.IGNORECASE)

        # convert <img> tags to LaTeX \includegraphics blocks
        soup = BeautifulSoup(text, "html.parser")
        for img in soup.find_all("img"):
            src = img.get("src", "")
            alt = img.get("alt", "")
            width = img.get("width")

            local_path = None
            # try downloading the image
            if src.startswith("http"):
                try:
                    file_name = os.path.basename(src.split("?")[0]) or "image.png"
                    if not re.search(r'\.(png|jpg|jpeg|gif|pdf|svg)$', file_name, re.IGNORECASE):
                        file_name += ".png"
                    local_path = os.path.join(images_dir, file_name)
                    r = requests.get(src, stream=True, timeout=10)
                    if r.status_code == 200:
                        with open(local_path, "wb") as f_img:
                            for chunk in r.iter_content(1024):
                                f_img.write(chunk)
                except Exception as e:
                    print(f"WARNING: could not download image {src}: {e}")
                files.download(local_path)
            latex_snippet = "% Image from: " + src + "\n\\begin{center}\n\\includegraphics"
            if width:
                try:
                    latex_snippet += f"[width={int(width)}pt]"
                except ValueError:
                    pass
            if local_path:
                latex_snippet += f"{{{local_path}}}\n"
            else:
                latex_snippet += f"{{{src}}}\n"

            if alt:
                latex_snippet += f"\\\\\\textit{{{alt}}}\n"
            latex_snippet += "\\end{center}"
            img.replace_with(latex_snippet)

        text = str(soup)

        # ensure blank lines before list items
        lines = text.splitlines()
        new_lines = []
        for i, line in enumerate(lines):
            if re.match(r'^\s*[-*+]\s', line):
                if i > 0 and lines[i-1].strip() != "":
                    new_lines.append("")
            new_lines.append(line)
        cell.source = "\n".join(new_lines)

with open(tmp_path + ".ipynb", "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

!jupyter nbconvert --to latex "{tmp_path}" --LatexPreprocessor.title "CUDA Exercises" --LatexPreprocessor.author_names "M. Ronzani"
time.sleep(1.5)

# replace escaped CUDA symbols
with open(tmp_path + ".tex", "r", encoding="utf-8") as f:
    tex_content = f.read()
tex_content = tex_content.replace("\\textgreater\\textgreater\\textgreater", ">{}>{}>")
tex_content = tex_content.replace("\\textless\\textless\\textless", "<{}<{}<")
with open(tmp_path + ".tex", "w", encoding="utf-8") as f:
    f.write(tex_content)

while not os.path.exists(tmp_path + ".tex"):
    print("Waiting for file write...")
    time.sleep(2)
files.download(tmp_path + ".tex")

os.remove(tmp_path + ".ipynb")

[NbConvertApp] Converting notebook /content/CUDA_tmp.ipynb to latex
[NbConvertApp] Writing 249826 bytes to /content/CUDA_tmp.tex


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>